# load packages

In [ ]:
import pandas as pd

In [ ]:
import os

In [ ]:
from datetime import datetime

In [ ]:
from zoneinfo import ZoneInfo

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
from scipy.stats import norm

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# read in input files

## zip code/SES

In [ ]:
# This query represents dataset "AOU v8 HF Clinical PGS" for domain "zip_code_socioeconomic" and was generated for All of Us Controlled Tier Dataset v8
dataset_62704170_zip_code_socioeconomic_sql = """
    SELECT
        observation.person_id,
        observation.observation_datetime,
        zip_code.zip3_as_string as zip_code,
        zip_code.fraction_assisted_income as assisted_income,
        zip_code.fraction_high_school_edu as high_school_education,
        zip_code.median_income,
        zip_code.fraction_no_health_ins as no_health_insurance,
        zip_code.fraction_poverty as poverty,
        zip_code.fraction_vacant_housing as vacant_housing,
        zip_code.deprivation_index,
        zip_code.acs as american_community_survey_year 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.zip3_ses_map` zip_code 
    JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
            ON CAST(SUBSTR(observation.value_as_string, 0, STRPOS(observation.value_as_string, '*') - 1) AS INT64) = zip_code.zip3  
    WHERE
        observation.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 )  
        UNION
        DISTINCT SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (44833557, 44819695, 1569178, 35207674, 44823110, 44831230, 44819696, 44824235, 44819692, 44821950, 44819693, 44824250, 35207673, 35207669, 44820856)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 0 
                        AND is_selectable = 1) 
                    AND is_standard = 0 )) criteria ) ) 
        AND observation_source_concept_id = 1585250 
        AND observation.value_as_string NOT LIKE 'Res%'"""

zip_code_socioeconomic = pd.read_gbq(
    dataset_62704170_zip_code_socioeconomic_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

print(len(zip_code_socioeconomic['person_id'].unique()))
zip_code_socioeconomic.head(5)

## demo

In [ ]:
# This query represents dataset "AOU v8 HF Clinical PGS" for domain "person" and was generated for All of Us Controlled Tier Dataset v8
dataset_62704170_person_sql = """
    SELECT
        person.person_id,
        person.birth_datetime as date_of_birth,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 )  
        UNION
        DISTINCT SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (44833557, 44819695, 1569178, 35207674, 44823110, 44831230, 44819696, 44824235, 44819692, 44821950, 44819693, 44824250, 35207673, 35207669, 44820856)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 0 
                        AND is_selectable = 1) 
                    AND is_standard = 0 )) criteria ) )"""

demo = pd.read_gbq(
    dataset_62704170_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

demo.head(5)

## ICD

In [ ]:
# This query represents dataset "AOU v8 HF Clinical PGS" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
dataset_62704170_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_start_datetime,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (1326492, 1326493, 1326600, 1326601, 1326602, 1326603, 1326604, 1326605, 1326606, 1326607, 1326608, 1326609, 1567956, 1567958, 1567959, 1567960, 1567964, 1567965, 1567966, 1567969, 1567971, 1569178, 1569179, 1569180, 1571687, 35206881, 35206882, 35207669, 35207673, 35207674, 35207792, 35207793, 37200198, 37200199, 37200200, 37200201, 37200202, 37200203, 37200204, 37200205, 37200206, 37200207, 37200208, 37200209, 37200210, 37200211, 37200212, 37200213, 37200214, 37200215, 37200216, 37200217, 37200218, 37200219, 37200220, 37200221, 37200222, 37200223, 37200224, 37200225, 37200227, 37200228, 37200229, 37200230, 37200232, 37200233, 37200234, 37200235, 37200237, 37200238, 37200239, 37200240, 37200242, 37200243, 37200244, 37200245, 37200246, 37200247, 37200248, 37200249, 37200251, 37200252, 37200253, 37200254, 44819500, 44819501, 44819502, 44819503, 44819504, 44819692, 44819693, 44819695, 44819696, 44820682, 44820683, 44820684, 44820685, 44820856,
 44820869, 44820870, 44821787, 44821950, 44822934, 44822935, 44822936, 44823110, 44823119, 44824071, 44824072, 44824073, 44824074, 44824235, 44824250, 44824251, 44825264, 44826459, 44826460, 44826461, 44826642, 44827615, 44827616, 44827617, 44827794, 44827795, 44827796, 44828793, 44828794, 44828795, 44829878, 44829879, 44829880, 44829881, 44829882, 44830086, 44831045, 44831046, 44831047, 44831230, 44831248, 44831249, 44831250, 44832190, 44832191, 44832192, 44832193, 44832194, 44832381, 44833365, 44833366, 44833367, 44833368, 44833557, 44833573, 44834548, 44834549, 44834732, 44835943, 44835944, 44836914, 44836915, 44836916, 44836917, 44836918, 45533019, 45533020, 45533021, 45533022, 45533023, 45533456, 45533457, 45537961, 45537962, 45542738, 45543182, 45547625, 45547626, 45547627, 45548022, 45552385, 45552386, 45557112, 45557113, 45561949, 45562355, 45566731, 45567180, 45567181, 45567896, 45576443, 45576878, 45581352, 45581353, 45581354, 45581355, 45582457, 45582458, 45586139, 45586140,
 45586587, 45586588, 45587292, 45587293, 45591027, 45591029, 45591030, 45591031, 45591469, 45595797, 45595798, 45595799, 45600641, 45600642, 45601038, 45605401, 45605402, 45605403, 45605404, 45605405, 45606547)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 )  
                UNION
                DISTINCT SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (44833557, 44819695, 1569178, 35207674, 44823110, 44831230, 44819696, 44824235, 44819692, 44821950, 44819693, 44824250, 35207673, 35207669, 44820856)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) )
            )
        ) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id"""

icd = pd.read_gbq(
    dataset_62704170_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

icd.head(5)

## labs

In [ ]:
# This query represents dataset "AOU v8 HF Clinical PGS" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v8
dataset_90496136_measurement_sql = """
    SELECT
        measurement.person_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.value_as_number,
        m_unit.concept_name as unit_concept_name,
        measurement.unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (1002597, 1175729, 1175898, 1175959, 1176311, 1176351, 1616654, 1618572, 21490674, 21490675, 21490678, 21490680, 21490851, 21490852, 21490853, 21492239, 21492240, 21492241, 21492391, 21494214, 3000261, 3000333, 3000404, 3000483, 3000637, 3000845, 3000940, 3001308, 3001318, 3001501, 3001506, 3001810, 3001975, 3001978, 3002009, 3002109, 3002240, 3002310, 3002574, 3002598, 3002666, 3002698, 3002785, 3003169, 3003201, 3003309, 3003403, 3003412, 3003435, 3003453, 3003767, 3003932, 3003994, 3004049, 3004077, 3004117, 3004209, 3004249, 3004410, 3004501, 3004529, 3004629, 3005030, 3005131, 3005478, 3005570, 3005606, 3005673, 3005787, 3006717, 3006887, 3006893, 3007034, 3007070, 3007263, 3007295, 3007332, 3007696, 3007943, 3008103, 3008286, 3008631, 3008770, 3008804, 3009261, 3009395, 3009397, 3009414, 3009435, 3009582, 3009718, 3009877, 3009966, 3010115, 3010300, 3010617, 3010956, 3011088, 3011161, 3011163, 3011367, 3011424, 3011498, 3011884, 3011972,
 3012278, 3012526, 3012789, 3012805, 3012888, 3013097, 3013104, 3013290, 3013473, 3013604, 3013668, 3013678, 3013702, 3013826, 3013940, 3014053, 3014305, 3014323, 3014600, 3014716, 3014737, 3015024, 3015178, 3015204, 3015544, 3015621, 3015739, 3015968, 3016083, 3016087, 3016159, 3016494, 3016699, 3016701, 3016798, 3016945, 3017188, 3017365, 3017490, 3017589, 3018056, 3018251, 3018336, 3018586, 3018592, 3018822, 3019013, 3019210, 3019240, 3019431, 3019464, 3019493, 3019575, 3019876, 3019955, 3019962, 3020044, 3020058, 3020107, 3020317, 3020345, 3020399, 3020491, 3020525, 3020632, 3020650, 3020869, 3021447, 3021513, 3021706, 3021737, 3021797, 3021860, 3021924, 3022038, 3022192, 3022268, 3022285, 3022314, 3022449, 3022548, 3022803, 3023024, 3023186, 3023243, 3023306, 3023386, 3023572, 3023574, 3023602, 3023752, 3023884, 3024047, 3024354, 3024629, 3024723, 3024762, 3025070, 3025202, 3025232, 3025398, 3025673, 3025839, 3025866, 3026041, 3026071, 3026300, 3026536, 3026677, 3026876, 3027198,
 3027315, 3027457, 3027468, 3027598, 3027801, 3027936, 3027939, 3027946, 3027997, 3028074, 3028247, 3028288, 3028437, 3028626, 3028646, 3028737, 3028803, 3028827, 3028944, 3029071, 3029133, 3029139, 3029246, 3029306, 3029335, 3030177, 3030260, 3030416, 3030437, 3030441, 3030997, 3031203, 3031266, 3031412, 3031465, 3031581, 3031681, 3031775, 3031973, 3032230, 3032260, 3032693, 3032719, 3032759, 3032771, 3032779, 3032987, 3033203, 3033254, 3033408, 3033638, 3033819, 3034207, 3034530, 3034639, 3034703, 3034962, 3035009, 3035125, 3035214, 3035250, 3035352, 3035398, 3035729, 3035759, 3035817, 3035856, 3035858, 3035899, 3036283, 3036406, 3036671, 3036807, 3036895, 3037014, 3037110, 3037187, 3037292, 3037524, 3037653, 3038071, 3038515, 3038553, 3038920, 3038988, 3039422, 3039426, 3039720, 3039851, 3039873, 3039896, 3039986, 3040151, 3040324, 3040820, 3041024, 3041253, 3041290, 3042145, 3042216, 3042443, 3042462, 3042637, 3043687, 3044242, 3044491, 3045156, 3045566, 3045700, 3045800, 3045807,
 3045989, 3046076, 3046405, 3046528, 3046549, 3046708, 3047111, 3048522, 3048865, 3049783, 3050630, 3050988, 3052202, 3052566, 3052598, 3053190, 3053286, 3053341, 36031550, 36032094, 36032229, 36032380, 36032416, 36032787, 36033366, 36033462, 36203185, 36204193, 36303264, 36304016, 36304326, 36304734, 36304833, 36305059, 36306043, 36716965, 37019625, 37019651, 37019808, 37019887, 37020015, 37020076, 37020303, 37020736, 37020823, 37020997, 37021192, 37021763, 37021905, 37021968, 37023397, 37024420, 37024641, 37024929, 37025002, 37025537, 37025901, 37025979, 37026560, 37026687, 37029357, 37029419, 37029979, 37030222, 37030259, 37030713, 37032253, 37032863, 37033044, 37033776, 37034437, 37034761, 37035033, 37035201, 37035650, 37037743, 37038593, 37038731, 37038784, 37039181, 37039374, 37040279, 37040418, 37040621, 37040744, 37040799, 37041042, 37042190, 37043257, 37043506, 37044332, 37045117, 37045382, 37045508, 37046668, 37047717, 37048053, 37048668, 37048990, 37051275, 37052309,
 37053001, 37053214, 37053398, 37054380, 37055462, 37055869, 37056686, 37056793, 37057730, 37058704, 37058713, 37059330, 37059768, 37060498, 37060543, 37060570, 37060933, 37062148, 37063613, 37064187, 37064430, 37065054, 37065592, 37067978, 37068379, 37069481, 37071197, 37072167, 37072239, 37073196, 37073389, 37073694, 37074494, 37076636, 37076649, 37078231, 37078343, 37078789, 37078832, 37393576, 37398559, 37399119, 37399654, 4012477, 4012479, 4016950, 4017078, 4017083, 4017760, 4017787, 4018315, 4018317, 4018318, 4019543, 4027514, 4032789, 4036846, 4041556, 4041697, 4041698, 4042059, 4042066, 4042759, 40482666, 40482677, 40485039, 4055667, 4055668, 4055695, 4060832, 4060833, 4060834, 4068414, 40757369, 40757503, 40757565, 40758413, 40758569, 40758736, 40759156, 40759279, 40759280, 40760809, 40761610, 40762249, 40762352, 40762636, 40762637, 40762638, 40765014, 4076704, 40768039, 40771054, 40772572, 40782589, 40782761, 40785880, 40789378, 40795740, 40795800, 4094447, 4094581, 4097882,
 4101713, 4108289, 4116187, 4120298, 4136579, 4143633, 4144235, 4149519, 4149883, 4151414, 4151548, 4152194, 4153111, 4154790, 4156660, 4182052, 4184637, 4191837, 4195214, 4195490, 4195503, 4197971, 4198718, 4198733, 4209122, 4218282, 4229586, 4230393, 4232915, 4236281, 4239021, 4245997, 4248524, 4248525, 4249006, 42537369, 4264765, 4268883, 42868678, 42868692, 4286945, 42869619, 42869628, 42869630, 4289453, 4292062, 4298391, 4298393, 4302410, 43054914, 43055485, 4331286, 4353851, 4354252, 4354254, 44786762, 44789315, 44789316, 44794807, 44812280, 44816672, 46235168, 46236280, 46236281, 46237026)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1) 
                OR  measurement_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (903106, 903107, 903109, 903110, 903113, 903114, 903115, 903116, 903118, 903124, 903129, 903130, 903132)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                measurement.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 )  
                UNION
                DISTINCT SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (44833557, 44819695, 1569178, 35207674, 44823110, 44831230, 44819696, 44824235, 44819692, 44821950, 44819693, 44824250, 35207673, 35207669, 44820856)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) )
            )
        ) measurement 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id"""

labs = pd.read_gbq(
    dataset_90496136_measurement_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

labs.head(5)

## observation

In [ ]:
# This query represents dataset "AOU v8 HF Clinical PGS" for domain "observation" and was generated for All of Us Controlled Tier Dataset v8
dataset_63000091_observation_sql = """
    SELECT
        observation.person_id,
        observation.observation_concept_id,
        o_standard_concept.concept_name as standard_concept_name,
        o_standard_concept.concept_code as standard_concept_code,
        o_standard_concept.vocabulary_id as standard_vocabulary,
        observation.observation_datetime,
        observation.value_as_number,
        observation.value_as_string,
        o_unit.concept_name as unit_concept_name,
        observation.observation_source_value,
        o_source_concept.concept_name as source_concept_name,
        o_source_concept.concept_code as source_concept_code,
        observation.unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
        WHERE
            (
                observation_concept_id IN (21492409, 21492410, 21492411, 21492412, 21492413, 21492414, 21492415, 21493945, 2617330, 2617331, 2617829, 2721640, 2721670, 3018063, 3043579, 38001390, 38001391, 4015401, 40217103, 40217330, 40217341, 4027529, 4031367, 4031370, 40483697, 4052051, 4053609, 4053842, 4063124, 40664614, 40664668, 40664817, 4069297, 4074782, 40766231, 40766232, 40766311, 40766360, 40766642, 40766646, 4083596, 4090847, 4098471, 4132133, 4151486, 4165523, 4165539, 4182465, 4182573, 4193014, 4195380, 4206526, 4213783, 4224317, 4237536, 42528763, 42528764, 4261613, 4267497, 4268546, 4270273, 4278461, 4278980, 4282779, 43021337, 4303438, 4304362, 43054909, 4305714, 4307293, 4310138, 4337839, 4338966, 43533208, 443359, 44786552, 44786668, 44792317, 44808782, 45890719, 762525, 763402, 763736, 765735, 801308, 915749) 
                OR  observation_source_concept_id IN (1585375, 1585741, 1585857, 1585860, 1585864, 1585867, 1585870, 1585873, 1585940, 1586159, 1586162, 1586166, 1586169, 1586174, 1586177, 1586182, 1586185, 40192384, 40192386, 40192400, 40192404, 40192410, 40192411, 40192412, 40192414, 40192417, 40192420, 40192431, 40192436, 40192437, 40192440, 40192456, 40192457, 40192458, 40192469, 40192476, 40192492, 40192493, 40192499, 40192500, 40192522)
            )  
            AND (
                observation.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 )  
                UNION
                DISTINCT SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (44833557, 44819695, 1569178, 35207674, 44823110, 44831230, 44819696, 44824235, 44819692, 44821950, 44819693, 44824250, 35207673, 35207669, 44820856)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) )
            )
        ) observation 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_standard_concept 
            ON observation.observation_concept_id = o_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_unit 
            ON observation.unit_concept_id = o_unit.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_source_concept 
            ON observation.observation_source_concept_id = o_source_concept.concept_id"""

observation = pd.read_gbq(
    dataset_63000091_observation_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

observation.head()

## survey

In [ ]:
# This query represents dataset "AOU v8 HF Clinical PGS" for domain "survey" and was generated for All of Us Controlled Tier Dataset v8
dataset_63000091_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (1585375, 1585741, 1585857, 1585860, 1585864, 1585867, 1585870, 1585873, 1585940, 1586159, 1586162, 1586166, 1586169, 1586174, 1586177, 1586182, 1586185, 40192384, 40192386, 40192400, 40192404, 40192410, 40192411, 40192412, 40192414, 40192417, 40192420, 40192431, 40192436, 40192437, 40192440, 40192456, 40192457, 40192458, 40192469, 40192476, 40192492, 40192493, 40192499, 40192500, 40192522)
        )  
        AND (
            answer.PERSON_ID IN (SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_whole_genome_variant = 1 )  
            UNION
            DISTINCT SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_whole_genome_variant = 1 ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (44833557, 44819695, 1569178, 35207674, 44823110, 44831230, 44819696, 44824235, 44819692, 44821950, 44819693, 44824250, 35207673, 35207669, 44820856)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 0 
                            AND is_selectable = 1) 
                        AND is_standard = 0 )) criteria ) )
        )"""

survey = pd.read_gbq(
    dataset_63000091_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

survey.head(5)

## PGS

In [ ]:
pgs = pd.read_csv('AOU/score/AOU_pgs.txt.gz', sep = '\t', usecols = ['IID', 'Z_norm2'], low_memory = False)
print(len(pgs.index))
pgs.head()

# clean demo df

## filter to males and females

In [ ]:
demo_sex = demo[demo['sex_at_birth'].isin(['Male', 'Female'])]
demo_sex['sex_at_birth'].unique()

## recode sex column

In [ ]:
demo_sex['sex_at_birth'] = demo_sex['sex_at_birth'].str.replace('Male','1')
demo_sex['sex_at_birth'] = demo_sex['sex_at_birth'].str.replace('Female','2')
demo_sex['sex_at_birth'].unique()

## rename sex column

In [ ]:
demo_sex.rename(columns = {'sex_at_birth' : 'SEX'}, inplace = True)
demo_sex.head()

## convert DOB column to datetime

In [ ]:
demo_sex['date_of_birth']=pd.to_datetime(demo_sex['date_of_birth'])

# clean icd df

## split hf and t2d dfs

In [ ]:
hf = icd[icd['source_concept_code'].str.contains('428|I50|I11|I13|404|402')]
print(hf['source_concept_code'].unique())
print(len(hf['person_id'].unique()))
hf.head()

In [ ]:
t2d = icd[icd['source_concept_code'].str.contains('250|E11|O24')]
print(t2d['source_concept_code'].unique())
print(len(t2d['person_id'].unique()))
t2d.head()

## check source vocabulary

In [ ]:
hf['source_vocabulary'].unique()

In [ ]:
t2d['source_vocabulary'].unique()

## rule of two

### create id lists

In [ ]:
hf_id_list = hf['person_id'].unique().tolist()
len(hf_id_list)

In [ ]:
t2d_id_list = t2d['person_id'].unique().tolist()
len(t2d_id_list)

### for loop to identify rule of 2 HF cases

In [ ]:
hf_two_instances = []
for id in hf_id_list:
    id_df = hf[hf['person_id'].isin([id])]
    id_df.drop_duplicates(inplace = True)
    id_df.reset_index(inplace = True, drop = True)
    if len(id_df.index) >= 2:
        hf_two_instances.append(id)

### for loop to identify rule of 2 T2D cases

In [ ]:
t2d_two_instances = []
for id in t2d_id_list:
    id_df = t2d[t2d['person_id'].isin([id])]
    id_df.drop_duplicates(inplace = True)
    id_df.reset_index(inplace = True, drop = True)
    if len(id_df.index) >= 2:
        t2d_two_instances.append(id)

### filter dfs to rule of two cases only

In [ ]:
hf_two = hf[hf['person_id'].isin(hf_two_instances)]
print(len(hf['person_id'].unique()))
print(len(hf_two['person_id'].unique()))
print(len(hf_two_instances))
hf_two.head()

In [ ]:
t2d_two = t2d[t2d['person_id'].isin(t2d_two_instances)]
print(len(t2d['person_id'].unique()))
print(len(t2d_two['person_id'].unique()))
print(len(t2d_two_instances))
t2d_two.head()

## drop extra columns

In [ ]:
hf_two.drop(columns = ['source_concept_code', 'source_vocabulary'], inplace = True)
hf_two.head()

In [ ]:
t2d_two.drop(columns = ['condition_start_datetime', 'source_concept_code', 'source_vocabulary'], inplace = True)
t2d_two.head()

## get first occurance for hf

In [ ]:
hf_two['condition_start_datetime'] = pd.to_datetime(hf_two['condition_start_datetime'])
hf_two.sort_values(by = ['person_id', 'condition_start_datetime'], inplace = True)
hf_two.drop_duplicates(subset = 'person_id', keep = 'first', inplace = True)
print(len(hf_two.index))
hf_two.head()

## drop duplicate ids for t2d

In [ ]:
t2d_two.drop_duplicates(inplace = True)
print(len(t2d_two.index))

## merge hf df with demo df

In [ ]:
hf_demo = hf_two.merge(demo_sex, on = 'person_id', how = 'inner')
print(len(hf_demo.index))
hf_demo.head()

## calculate age at first occurance

In [ ]:
hf_demo['AGE'] = hf_demo['condition_start_datetime'] - hf_demo['date_of_birth']
hf_demo['AGE'] = hf_demo['AGE'].astype(str).str.replace(r' .*', '', regex = True)
hf_demo['AGE'] = hf_demo['AGE'].astype(float)
hf_demo['AGE'] = hf_demo['AGE']/365.25
hf_demo.head()

## drop DOB and condition start time columns from hf df

In [ ]:
hf_demo.drop(columns = ['date_of_birth'], inplace = True)
hf_demo.rename(columns = {'condition_start_datetime' : 'HF_DATE'}, inplace = True)
hf_demo.head()

## add case columns

In [ ]:
hf_demo['HF'] = 1
hf_demo.head()

In [ ]:
t2d_two['T2D'] = 1
t2d_two.head()

# create hf controls dataframe

## remove individuals with hf icd code

In [ ]:
hf_controls = demo_sex[~demo_sex['person_id'].isin(hf['person_id'])]
print(len(hf_controls.index))
hf_controls.head()

## calculate age at data release

In [ ]:
hf_controls['HF_DATE'] = pd.to_datetime('2023-10-01')
hf_controls['AGE'] = hf_controls['HF_DATE'] - hf_controls['date_of_birth'].dt.tz_localize(None)
hf_controls['AGE'] = hf_controls['AGE'].astype(str).str.replace(' days', '')
hf_controls['AGE'] = hf_controls['AGE'].astype(float)
hf_controls['AGE'] = hf_controls['AGE'] / 365.25
hf_controls.head()

## drop dob column

In [ ]:
hf_controls.drop(columns = ['date_of_birth'], inplace = True)
hf_controls.head()

## add HF column

In [ ]:
hf_controls['HF'] = 0
hf_controls.head()

# merge hf cases and controls

## merge

In [ ]:
hf_case_control = pd.concat([hf_demo, hf_controls], axis = 0)
print(len(hf_case_control.index))
print(hf_case_control['HF'].value_counts())
hf_case_control.head()

# merge w t2d df

## merge

In [ ]:
hf_t2d = hf_case_control.merge(t2d_two, on = 'person_id', how = 'left')
hf_t2d['T2D'] = hf_t2d['T2D'].fillna(0)
print(len(hf_t2d.index))
print(hf_t2d['T2D'].value_counts())
hf_t2d.head()

## identify individuals w one t2d icd code and set column as missing

In [ ]:
hf_t2d_one = hf_t2d[hf_t2d['person_id'].isin(t2d['person_id'])]
hf_t2d_one = hf_t2d_one[~hf_t2d_one['person_id'].isin(t2d_two['person_id'])]
print(len(hf_t2d_one.index))
hf_t2d_one.head()

In [ ]:
hf_t2d_one['T2D'] = hf_t2d_one['T2D'].replace(0.0, np.nan)
print(hf_t2d_one['T2D'].unique())
hf_t2d_one.head()

In [ ]:
hf_t2d_no_one = hf_t2d[~hf_t2d['person_id'].isin(hf_t2d_one['person_id'])]
print(len(hf_t2d_no_one.index))

In [ ]:
hf_t2d_fixed = pd.concat([hf_t2d_no_one, hf_t2d_one], axis = 0)
print(len(hf_t2d_fixed.index))
print(hf_t2d_fixed['T2D'].value_counts(dropna=False))
hf_t2d_fixed.head()

## remove decimal place from t2d column

In [ ]:
hf_t2d_fixed['T2D'] = hf_t2d_fixed['T2D'].astype('Int64')
print(hf_t2d_fixed['T2D'].value_counts(dropna = False))
hf_t2d_fixed.head()

# clean labs df

## create median absolute deviance function

In [ ]:
def mad_filter(df, threshold = 5):
    values = df['value_as_number'].dropna()
    
    # Compute median and MAD
    median = values.median()
    mad = np.median(np.abs(values - median))
    
    # Define limits
    lower_limit = median - threshold * mad
    upper_limit = median + threshold * mad

    # Filter and add top/bottom limits
    filtered_df = df[(df['value_as_number'] >= lower_limit) & (df['value_as_number'] <= upper_limit)].copy()

    return filtered_df

## subset HF dateframe w just date and id

In [ ]:
hf_date = hf_t2d_fixed[['person_id', 'HF_DATE']]

## triglyerides

### filter by name of measurement

In [ ]:
trig = labs[labs['standard_concept_name'].str.contains('triglyceride', case = False)]
print(trig['standard_concept_name'].unique())

In [ ]:
trig = trig[~trig['standard_concept_name'].str.contains('Cholesterol|VLDL|Chylomicrons|LDL|IDL|Percentile|Lipids|Pleural|Peritoneal|Specimen|HDL|DBS|Fluid|Blood|Triglycerides', case = False)]
print(trig['standard_concept_name'].unique())

In [ ]:
print(len(trig.index))
print(len(trig['person_id'].unique()))

### apply MAD function

In [ ]:
trig_normal = mad_filter(trig)
print(len(trig_normal['person_id'].unique()))
print(trig_normal['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
trig_normal.head()

### remove negative and zero values

In [ ]:
trig_positive = trig_normal[trig_normal['value_as_number'] > 0]
print(len(trig_positive['person_id'].unique()))
print(trig_positive['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### filter by date

In [ ]:
trig_date = trig_positive.merge(hf_date, on = 'person_id', how = 'inner')
trig_date['DIFF'] = pd.to_datetime(trig_date['HF_DATE'], utc = True) - trig_date['measurement_datetime']
trig_date['DIFF'] = trig_date['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
trig_date['DIFF'] = trig_date['DIFF'].astype(float)
print(len(trig_sub.index))
print(len(trig_date.index))
trig_date.head()

In [ ]:
trig_first = trig_date[trig_date['DIFF'] >= 0]
trig_first.sort_values(by = ['person_id', 'DIFF'], inplace = True)
print(len(trig_first.index))
trig_first.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(trig_first.index))
print(trig_first['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### remove extra columns and rename triglyeride column

In [ ]:
trig_sub = trig_first[['person_id', 'value_as_number']]
trig_sub.rename(columns = {'value_as_number' : 'TRIG'}, inplace = True)
trig_sub.head()

### merge with hf df

In [ ]:
hf_trig = hf_t2d_fixed.merge(trig_sub, on = 'person_id', how = 'left')
print(len(hf_trig.index))
print(len(hf_trig[hf_trig['TRIG'].isna() == True ]))
hf_trig.head()

## ldl

### filter by measurement name

In [ ]:
ldl = labs[labs['standard_concept_name'].str.contains('low|density|lipoprotein|ldl|cholesterol', case = False)]
ldl['standard_concept_name'].unique()

In [ ]:
ldl = ldl[~ldl['standard_concept_name'].str.contains('HDL|VLDL|subparticle|alpha|Percentile|Triglyceride|mutation|^High|oxidized|dense|total|narrow|fluid|1|2|3|4', case = False)]
ldl['standard_concept_name'].unique()

In [ ]:
ldl = ldl[~ldl['standard_concept_name'].str.contains('measurement|pre|Calculated|lipid|DBS|Identifier', case = False)]
ldl['standard_concept_name'].unique()

In [ ]:
print(len(ldl['person_id'].unique()))
print(ldl['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### apply MAD

In [ ]:
ldl_normal = mad_filter(ldl)
print(len(ldl_normal['person_id'].unique()))
print(ldl_normal['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
ldl_normal.head()

### remove negative and zero labs

In [ ]:
ldl_positive = ldl_normal[ldl_normal['value_as_number'] > 0]
print(len(ldl_positive['person_id'].unique()))
print(ldl_positive['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### filter by date

In [ ]:
ldl_date = ldl_positive.merge(hf_date, on = 'person_id', how = 'inner')
ldl_date['DIFF'] = pd.to_datetime(ldl_date['HF_DATE'], utc = True) - ldl_date['measurement_datetime']
ldl_date['DIFF'] = ldl_date['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
ldl_date['DIFF'] = ldl_date['DIFF'].astype(float)

ldl_first = ldl_date[ldl_date['DIFF'] >= 0]
ldl_first.sort_values(by = ['person_id', 'DIFF'], inplace = True)
ldl_first.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(ldl_first.index))
print(ldl_first['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### remove extra columns & rename

In [ ]:
ldl_sub = ldl_first[['person_id', 'value_as_number']]
ldl_sub.rename(columns = {'value_as_number' : 'LDL'}, inplace = True)
ldl_sub.head()

### merge w hf df

In [ ]:
hf_ldl = hf_trig.merge(ldl_sub, on = 'person_id', how = 'left')
print(len(hf_ldl.index))
print(len(hf_ldl[hf_ldl['LDL'].isna() == True ]))
hf_ldl.head()

## hdl

### filter by measurement name

In [ ]:
hdl = labs[labs['standard_concept_name'].str.contains('high|density|lipoprotein|hdl|cholesterol', case = False)]
hdl['standard_concept_name'].unique()

In [ ]:
hdl = hdl[~hdl['standard_concept_name'].str.contains('LDL|Triglyceride|measurement|beta|Ratio|Percentile|total|non|3|subparticle|2|length|Presence|electroph', case = False)]
hdl['standard_concept_name'].unique()

In [ ]:
print(hdl['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
print(len(hdl['person_id'].unique()))

### apply mad

In [ ]:
hdl_normal = mad_filter(hdl)
print(len(hdl_normal['person_id'].unique()))
print(hdl_normal['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
hdl_normal.head()

### remove values less than zero

In [ ]:
hdl_positive = hdl_normal[hdl_normal['value_as_number'] > 0]
print(len(hdl_positive['person_id'].unique()))
print(hdl_positive['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### filter by date

In [ ]:
hdl_date = hdl_positive.merge(hf_date, on = 'person_id', how = 'inner')
hdl_date['DIFF'] = pd.to_datetime(hdl_date['HF_DATE'], utc = True) - hdl_date['measurement_datetime']
hdl_date['DIFF'] = hdl_date['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
hdl_date['DIFF'] = hdl_date['DIFF'].astype(float)

hdl_first = hdl_date[hdl_date['DIFF'] >= 0]
hdl_first.sort_values(by = ['person_id', 'DIFF'], inplace = True)
hdl_first.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(hdl_first.index))
print(hdl_first['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### remove extra columns and rename

In [ ]:
hdl_sub = hdl_first[['person_id', 'value_as_number']]
hdl_sub.rename(columns = {'value_as_number' : 'HDL'}, inplace = True)
hdl_sub.head()

### merge w hf df

In [ ]:
hf_hdl = hf_ldl.merge(hdl_sub, on = 'person_id', how = 'left')
print(len(hf_hdl.index))
print(len(hf_hdl[hf_hdl['HDL'].isna() == True ]))
hf_hdl.head()

## glucose

### filter by measurement name

In [ ]:
glucose = labs[labs['standard_concept_name'].str.contains('glucose', case = False)]
glucose['standard_concept_name'].unique()

In [ ]:
glucose = glucose[~glucose['standard_concept_name'].str.contains('Enzymatic|DBS|Presence|measurement|Cerebral|time|tolerance|Laboratory|level|Ratio|PO|Insulin|meter|Self-monitoring|fluid|qualitative|Specimen|Entitic|test|Urine|Blood', case = False)]
glucose['standard_concept_name'].unique()

In [ ]:
print(glucose['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
print(len(glucose['person_id'].unique()))

### filter by non-fasting- higher sample size

In [ ]:
not_fasting = glucose[glucose['standard_concept_name'].isin(['Glucose [Mass/volume] in Serum or Plasma',
                                                                       'Glucose [Moles/volume] in Serum or Plasma'])]
print(len(not_fasting['person_id'].unique()))
print(not_fasting['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### apply MAD filter

In [ ]:
glucose_normal = mad_filter(not_fasting)
print(len(glucose_normal['person_id'].unique()))
print(glucose_normal['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
glucose_normal.head()

### filter by date

In [ ]:
glucose_date = glucose_normal.merge(hf_date, on = 'person_id', how = 'inner')
glucose_date['DIFF'] = pd.to_datetime(glucose_date['HF_DATE'], utc = True) - glucose_date['measurement_datetime']
glucose_date['DIFF'] = glucose_date['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
glucose_date['DIFF'] = glucose_date['DIFF'].astype(float)

glucose_first = glucose_date[glucose_date['DIFF'] >= 0]
glucose_first.sort_values(by = ['person_id', 'DIFF'], inplace = True)
glucose_first.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(glucose_first.index))
print(glucose_first['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### remove extra columns and rename

In [ ]:
glucose_sub = glucose_first[['person_id', 'value_as_number']]
glucose_sub.rename(columns = {'value_as_number' : 'GLUCOSE'}, inplace = True)
glucose_sub.head()

### merge w hf df

In [ ]:
hf_glucose = hf_hdl.merge(glucose_sub, on = 'person_id', how = 'left')
print(len(hf_glucose.index))
print(len(hf_glucose[hf_glucose['GLUCOSE'].isna() == True ]))
hf_glucose.head()

## hba1c

### filter by measurement name

In [ ]:
hba1c = labs[labs['standard_concept_name'].str.contains('hba1c|hemoglobin|a1c', case = False)]
hba1c['standard_concept_name'].unique()

In [ ]:
hba1c = hba1c[hba1c['standard_concept_name'].str.contains('A1c/')]
hba1c['standard_concept_name'].unique()

In [ ]:
print(hba1c['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
print(len(hba1c['person_id'].unique()))

### apply MAD

In [ ]:
hba1c_normal = mad_filter(hba1c)
print(len(hba1c_normal['person_id'].unique()))
print(hba1c_normal['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
hba1c_normal.head()

### filter by date

In [ ]:
hba1c_date = hba1c_normal.merge(hf_date, on = 'person_id', how = 'inner')
hba1c_date['DIFF'] = pd.to_datetime(hba1c_date['HF_DATE'], utc = True) - hba1c_date['measurement_datetime']
hba1c_date['DIFF'] = hba1c_date['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
hba1c_date['DIFF'] = hba1c_date['DIFF'].astype(float)

hba1c_first = hba1c_date[hba1c_date['DIFF'] >= 0]
hba1c_first.sort_values(by = ['person_id', 'DIFF'], inplace = True)
hba1c_first.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(hba1c_first.index))
print(hba1c_first['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### remove extra columns and rename

In [ ]:
hba1c_sub = hba1c_first[['person_id', 'value_as_number']]
hba1c_sub.rename(columns = {'value_as_number' : 'HbA1c'}, inplace = True)
hba1c_sub.head()

### merge w hf df

In [ ]:
hf_hba1c = hf_glucose.merge(hba1c_sub, on = 'person_id', how = 'left')
print(len(hf_hba1c.index))
print(len(hf_hba1c[hf_hba1c['HbA1c'].isna() == True ]))
hf_hba1c.head()

## systolic blood pressure

### filter by measurement name

In [ ]:
systolic = labs[labs['standard_concept_name'].str.contains('systolic|pressure', case = False)]
systolic['standard_concept_name'].unique()

In [ ]:
systolic = systolic[~systolic['standard_concept_name'].str.contains('diastolic|oxygen|carbon|invasive|Cuff|venous|Arterial|Type|ventricular|toe|Mean|Artery|Atrial|panel|Aorta|taking', case = False)]
systolic['standard_concept_name'].unique()

In [ ]:
systolic = systolic[~systolic['standard_concept_name'].isin(['Sitting blood pressure',
                                                             'Standing blood pressure'])]
systolic['standard_concept_name'].unique()

In [ ]:
print(systolic['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
print(len(systolic['person_id'].unique()))

### apply MAD

In [ ]:
systolic_normal = mad_filter(systolic)
print(len(systolic_normal['person_id'].unique()))
print(systolic_normal['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
systolic_normal.head()

### filter by date

In [ ]:
systolic_date = systolic_normal.merge(hf_date, on = 'person_id', how = 'inner')
systolic_date['DIFF'] = pd.to_datetime(systolic_date['HF_DATE'], utc = True) - systolic_date['measurement_datetime']
systolic_date['DIFF'] = systolic_date['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
systolic_date['DIFF'] = systolic_date['DIFF'].astype(float)

systolic_first = systolic_date[systolic_date['DIFF'] >= 0]
systolic_first.sort_values(by = ['person_id', 'DIFF'], inplace = True)
systolic_first.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(systolic_first.index))
print(systolic_first['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### remove extra columns and rename

In [ ]:
systolic_sub = systolic_first[['person_id', 'value_as_number']]
systolic_sub.rename(columns = {'value_as_number' : 'SBP'}, inplace = True)
systolic_sub.head()

### merge w hf df

In [ ]:
hf_systolic = hf_hba1c.merge(systolic_sub, on = 'person_id', how = 'left')
print(len(hf_systolic.index))
print(len(hf_systolic[hf_systolic['SBP'].isna() == True ]))
hf_systolic.head()

## diastolic blood pressure

### filter by measurement name

In [ ]:
diastolic = labs[labs['standard_concept_name'].str.contains('diastolic', case = False)]
diastolic['standard_concept_name'].unique()

In [ ]:
diastolic = diastolic[~diastolic['standard_concept_name'].str.contains('artery|systolic|arterial|Invasive', case = False)]
diastolic['standard_concept_name'].unique()

In [ ]:
print(diastolic['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
print(len(diastolic['person_id'].unique()))

### apply mad filter

In [ ]:
diastolic_normal = mad_filter(diastolic)
print(len(diastolic_normal['person_id'].unique()))
print(diastolic_normal['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
diastolic_normal.head()

### filter by date

In [ ]:
diastolic_date = diastolic_normal.merge(hf_date, on = 'person_id', how = 'inner')
diastolic_date['DIFF'] = pd.to_datetime(diastolic_date['HF_DATE'], utc = True) - diastolic_date['measurement_datetime']
diastolic_date['DIFF'] = diastolic_date['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
diastolic_date['DIFF'] = diastolic_date['DIFF'].astype(float)

diastolic_first = diastolic_date[diastolic_date['DIFF'] >= 0]
diastolic_first.sort_values(by = ['person_id', 'DIFF'], inplace = True)
diastolic_first.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(diastolic_first.index))
print(diastolic_first['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### remove extra columns and rename

In [ ]:
diastolic_sub = diastolic_first[['person_id', 'value_as_number']]
diastolic_sub.rename(columns = {'value_as_number' : 'DBP'}, inplace = True)
diastolic_sub.head()

### merge w hf df

In [ ]:
hf_diastolic = hf_systolic.merge(diastolic_sub, on = 'person_id', how = 'left')
print(len(hf_diastolic.index))
print(len(hf_diastolic[hf_diastolic['DBP'].isna() == True ]))
hf_diastolic.head()

## bmi

### filter by measurement name

In [ ]:
bmi = labs[labs['standard_concept_name'].str.contains('BMI|Body', case = False)]
bmi['standard_concept_name'].unique()

In [ ]:
bmi = bmi[~bmi['standard_concept_name'].str.contains('fluid', case = False)]
bmi['standard_concept_name'].unique()

In [ ]:
print(bmi['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
print(len(bmi['person_id'].unique()))

### apply MAD filter

In [ ]:
bmi_normal = mad_filter(bmi)
print(len(bmi_normal['person_id'].unique()))
print(bmi_normal['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
bmi_normal.head()

### filter by date

In [ ]:
bmi_date = bmi_normal.merge(hf_date, on = 'person_id', how = 'inner')
bmi_date['DIFF'] = pd.to_datetime(bmi_date['HF_DATE'], utc = True) - bmi_date['measurement_datetime']
bmi_date['DIFF'] = bmi_date['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
bmi_date['DIFF'] = bmi_date['DIFF'].astype(float)

bmi_first = bmi_date[bmi_date['DIFF'] >= 0]
bmi_first.sort_values(by = ['person_id', 'DIFF'], inplace = True)
bmi_first.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(bmi_first.index))
print(bmi_first['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))

### remove extra columns and rename

In [ ]:
bmi_sub = bmi_first[['person_id', 'value_as_number']]
bmi_sub.rename(columns = {'value_as_number' : 'BMI'}, inplace = True)
bmi_sub.head()

### merge w hf df

In [ ]:
hf_bmi = hf_diastolic.merge(bmi_sub, on = 'person_id', how = 'left')
print(len(hf_bmi.index))
print(len(hf_bmi[hf_bmi['BMI'].isna() == True ]))
hf_bmi.head()

# clean observations data

## neighborhood

### encoding
**My neighborhood has several free or low-cost recreation facilities, such as parks, walking trails, bike paths, recreation centers, playgrounds, public swimming pools, etc [PhenX]**
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15235-7' : 2 (Somewhat agree)
- 'LA15236-5' : 0 (Strongly disagree)
- 'LA15234-0' : 1 (Somewhat disagree)
- 'PMI_Skip' : np.nan (Missing)

**How much you agree or disagree that there is too much drug use in your neighborhood?**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15773-7' : 2 (Disagree)
- 'LA15774-5' : 1 (Agree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15237-3' : 0 (Strongly agree)

**My neighborhood is safe from crime [PhenX]**
- 'LA15774-5' : 2 (Agree)
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15773-7' : 1 (Disagree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15236-5' : 0 (Strongly disagree)

**People in this neighborhood can be trusted [PhenX]**
- 'LA15774-5' : 3 (Agree)
- 'LA15237-3' : 4 (Strongly agree)
- 'LA15773-7' : 1 (Disagree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15236-5' : 0 (Strongly disagree)
- 'LA15240-7' : 2 (Neither agree nor disagree)

**How much you agree or disagree that there are lot of abandoned buildings in your neighborhood?**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15773-7' : 2 (Disagree)
- 'LA15774-5' : 1 (Agree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15237-3' : 0 (Strongly agree)

**How much you agree or disagree that there is too much alcohol use in your neighborhood?**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15773-7' : 2 (Disagree)
- 'LA15774-5' : 1 (Agree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15237-3' : 0 (Strongly agree)

**How much you agree or disagree that vandalism is common in your neighborhood?**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15773-7' : 2 (Disagree)
- 'LA15774-5' : 1 (Agree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15237-3' : 0 (Strongly agree)

**There are sidewalks on most of the streets in my neighborhood [PhenX]**
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15236-5' : 0 (Strongly disagree)
- 'LA15235-7' : 2 (Somewhat agree)
- 'LA15234-0' : 1 (Somewhat disagree)
- 'SDOH_9' : np.nan (don't know what this means so setting it as missing)
- 'PMI_Skip' : np.nan (Missing)

**There are facilities to bicycle in or near my neighborhood, such as special lanes, separate paths or trails, shared use paths for cycles and pedestrians [PhenX]**
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15235-7' : 2 (Somewhat agree)
- 'LA15236-5' : 0 (Strongly disagree)
- 'LA15234-0' : 1 (Somewhat disagree)
- 'PMI_Skip' : np.nan (Missing)
- 'SDOH_9' : np.nan (don't know what this means so setting it as missing)

**How much you agree or disagree that your neighborhood is clean?**
- 'LA15774-5' : 2 (Agree)
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15773-7' : 1 (Disagree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15236-5' : 0 (Strongly disagree)

**How much you agree or disagree that in your neighborhood people watch out for each other?**
- 'LA15774-5' : 2 (Agree)
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15773-7' : 1 (Disagree)
- 'LA15236-5' : 0 (Strongly disagree)
- 'PMI_Skip' : np.nan (Missing)

**What is the main type of housing in your neighborhood [PhenX]**
- 'LA15229-0' : 4 (Detached single-family housing)
- 'LA15231-6' : 2 (Mix of single-family residences and townhouses, row houses, apartments, or condos)
- 'LA30308-3' : 3 (Duplex/townhouse)
- 'LA15232-4' : 1 (Apartments or condos of 4-12 stories)
- 'LA15233-2' : 0 (Apartments or condos of more than 12 stories)
- 'PMI_Skip' : np.nan (Missing)

**People in this neighborhood generally don't get along with each other [PhenX]**
- 'LA15774-5' : 1 (Agree)
- 'LA15237-3' : 0 (Strongly agree)
- 'LA15240-7' : 2 (Neither agree nor disagree)
- 'LA15773-7' : 3 (Disagree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15236-5' : 4 (Strongly disagree)

**How much you agree or disagree that there is a lot of graffiti in your neighborhood?**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15773-7' : 2 (Disagree)
- 'LA15774-5' : 1 (Agree)
- 'LA15237-3' : 0 (Strongly agree)
- 'PMI_Skip' : np.nan (Missing)

**How much you agree or disagree that your neighborhood is noisy?**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15773-7' : 2 (Disagree)
- 'LA15774-5' : 1 (Agree)
- 'LA15237-3' : 0 (Strongly agree)
- 'PMI_Skip' : np.nan (Missing)

**The crime rate in my neighborhood makes it unsafe to go on walks at night [PhenX]**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15234-0' : 2 (Somewhat disagree)
- 'LA15235-7' : 1 (Somewhat agree)
- 'LA15237-3' : 0 (Strongly agree)
- 'PMI_Skip' : np.nan (Missing)

**How much you agree or disagree that people in your neighborhood take good care of their houses and apartments?**
- 'LA15774-5' : 2 (Agree)
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15773-7' : 1 (Disagree)
- 'LA15236-5' : 0 (Strongly disagree)
- 'PMI_Skip' : np.nan (Missing)

**How much you agree or disagree that there is a lot of crime in your neighborhood?**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15773-7' : 2 (Disagree)
- 'LA15774-5' : 1 (Agree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15237-3' : 0 (Strongly agree)

**The crime rate in my neighborhood makes it unsafe to go on walks during the day. Would you say that you...**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15234-0' : 2 (Somewhat disagree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15235-7' : 1 (Somewhat agree)
- 'LA15237-3' : 0 (Strongly agree)

**How much you agree or disagree that people in your neighborhood share the same values?**
- 'LA15240-7' : 2 (Neither agree nor disagree)
- 'LA15774-5' : 3 (Agree)
- 'LA15237-3' : 4 (Strongly agree)
- 'LA15773-7' : 1 (Disagree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15236-5' : 0 (Strongly disagree)

**How much you agree or disagree that there are too many people hanging around on the streets near your home?**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15773-7' : 2 (Disagree)
- 'LA15774-5' : 1 (Agree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15237-3' : 0 (Strongly agree)

**How much you agree or disagree that you are always having trouble with your neighbors?**
- 'LA15236-5' : 3 (Strongly disagree)
- 'LA15773-7' : 2 (Disagree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15774-5' : 1 (Agree)
- 'LA15237-3' : 0 (Strongly agree)

**Many shops, stores, markets, or other places to buy things I need are within easy walking distance of my home [PhenX]**
- 'LA15236-5' : 0 (Strongly disagree)
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15235-7' : 2 (Somewhat agree)
- 'LA15234-0' : 1 (Somewhat disagree)
- 'PMI_Skip' : np.nan (Missing)

**It is within a 10-15 minute walk to a transit stop, such as bus, train, trolley, or tram, from my home [PhenX]**
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15236-5' : 0 (Strongly disagree)
- 'LA15235-7' : 2 (Somewhat agree)
- 'LA15234-0' : 1 (Somewhat disagree)
- 'PMI_Skip' : np.nan (Missing)

### filter oberservations df to neighborhood

In [ ]:
neighbor_obs = observation[observation['standard_concept_name'].str.contains('neighbor|transit|store|street')]
neighbor_obs['standard_concept_name'].unique()

### free amenities

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['My neighborhood has several free or low-cost recreation facilities, such as parks, walking trails, bike paths, recreation centers, playgrounds, public swimming pools, etc [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_FREE_AMENITIES'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_FREE_AMENITIES']]
neighbor_obs_sub['NEIGHBORHOOD_FREE_AMENITIES'] = neighbor_obs_sub['NEIGHBORHOOD_FREE_AMENITIES'].replace({'LA15237-3' : 3,
                                                                                                         'LA15235-7' : 2,
                                                                                                         'LA15236-5' : 0,
                                                                                                         'LA15234-0' : 1,
                                                                                                         'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_FREE_AMENITIES'] = neighbor_obs_sub['NEIGHBORHOOD_FREE_AMENITIES'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_FREE_AMENITIES'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_sub

### drug use

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that there is too much drug use in your neighborhood?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_DRUG_USE'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_DRUG_USE']]
neighbor_obs_sub['NEIGHBORHOOD_DRUG_USE'] = neighbor_obs_sub['NEIGHBORHOOD_DRUG_USE'].replace({'LA15236-5' : 3,
                                                                                               'LA15773-7' : 2,
                                                                                               'LA15774-5' : 1,
                                                                                               'PMI_Skip' : np.nan,
                                                                                               'LA15237-3' : 0})
neighbor_obs_sub['NEIGHBORHOOD_DRUG_USE'] = neighbor_obs_sub['NEIGHBORHOOD_DRUG_USE'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_DRUG_USE'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### safe from crime

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['My neighborhood is safe from crime [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_SAFE_CRIME'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_SAFE_CRIME']]
neighbor_obs_sub['NEIGHBORHOOD_SAFE_CRIME'] = neighbor_obs_sub['NEIGHBORHOOD_SAFE_CRIME'].replace({'LA15774-5' : 2,
                                                                                                   'LA15237-3' : 3,
                                                                                                   'LA15773-7' : 1,
                                                                                                   'PMI_Skip' : np.nan,
                                                                                                   'LA15236-5' : 0})
neighbor_obs_sub['NEIGHBORHOOD_SAFE_CRIME'] = neighbor_obs_sub['NEIGHBORHOOD_SAFE_CRIME'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_SAFE_CRIME'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### neighborhood trust

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['People in this neighborhood can be trusted [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_TRUST'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_TRUST']]
neighbor_obs_sub['NEIGHBORHOOD_TRUST'] = neighbor_obs_sub['NEIGHBORHOOD_TRUST'].replace({'LA15774-5' : 3,
                                                                                         'LA15237-3' : 4,
                                                                                         'LA15773-7' : 1,
                                                                                         'PMI_Skip' : np.nan,
                                                                                         'LA15236-5' : 0,
                                                                                         'LA15240-7' : 2})
neighbor_obs_sub['NEIGHBORHOOD_TRUST'] = neighbor_obs_sub['NEIGHBORHOOD_TRUST'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_TRUST'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### abandoned buildings

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that there are lot of abandoned buildings in your neighborhood?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_BUILDING'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_BUILDING']]
neighbor_obs_sub['NEIGHBORHOOD_BUILDING'] = neighbor_obs_sub['NEIGHBORHOOD_BUILDING'].replace({'LA15236-5' : 3,
                                                                                               'LA15773-7' : 2,
                                                                                               'LA15774-5' : 1,
                                                                                               'PMI_Skip' : np.nan,
                                                                                               'LA15237-3' : 0})
neighbor_obs_sub['NEIGHBORHOOD_BUILDING'] = neighbor_obs_sub['NEIGHBORHOOD_BUILDING'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_BUILDING'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### alcohol use

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that there is too much alcohol use in your neighborhood?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_ALCOHOL'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_ALCOHOL']]
neighbor_obs_sub['NEIGHBORHOOD_ALCOHOL'] = neighbor_obs_sub['NEIGHBORHOOD_ALCOHOL'].replace({'LA15236-5' : 3,
                                                                                             'LA15773-7' : 2,
                                                                                             'LA15774-5' : 1,
                                                                                             'PMI_Skip' : np.nan,
                                                                                             'LA15237-3' : 0})
neighbor_obs_sub['NEIGHBORHOOD_ALCOHOL'] = neighbor_obs_sub['NEIGHBORHOOD_ALCOHOL'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_ALCOHOL'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### vandalism

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that vandalism is common in your neighborhood?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_VANDALISM'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_VANDALISM']]
neighbor_obs_sub['NEIGHBORHOOD_VANDALISM'] = neighbor_obs_sub['NEIGHBORHOOD_VANDALISM'].replace({'LA15236-5' : 3,
                                                                                             'LA15773-7' : 2,
                                                                                             'LA15774-5' : 1,
                                                                                             'PMI_Skip' : np.nan,
                                                                                             'LA15237-3' : 0})
neighbor_obs_sub['NEIGHBORHOOD_VANDALISM'] = neighbor_obs_sub['NEIGHBORHOOD_VANDALISM'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_VANDALISM'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### sidewalks

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['There are sidewalks on most of the streets in my neighborhood [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_SIDEWALK'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_SIDEWALK']]
neighbor_obs_sub['NEIGHBORHOOD_SIDEWALK'] = neighbor_obs_sub['NEIGHBORHOOD_SIDEWALK'].replace({'LA15237-3' : 3,
                                                                                               'LA15236-5' : 0,
                                                                                               'LA15235-7' : 2,
                                                                                               'LA15234-0' : 1,
                                                                                               'SDOH_9' : np.nan,
                                                                                               'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_SIDEWALK'] = neighbor_obs_sub['NEIGHBORHOOD_SIDEWALK'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_SIDEWALK'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### bike

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['There are facilities to bicycle in or near my neighborhood, such as special lanes, separate paths or trails, shared use paths for cycles and pedestrians [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_BIKE'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_BIKE']]
neighbor_obs_sub['NEIGHBORHOOD_BIKE'] = neighbor_obs_sub['NEIGHBORHOOD_BIKE'].replace({'LA15237-3' : 3,
                                                                                               'LA15236-5' : 0,
                                                                                               'LA15235-7' : 2,
                                                                                               'LA15234-0' : 1,
                                                                                               'SDOH_9' : np.nan,
                                                                                               'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_BIKE'] = neighbor_obs_sub['NEIGHBORHOOD_BIKE'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_BIKE'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### clean

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that your neighborhood is clean?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_CLEAN'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_CLEAN']]
neighbor_obs_sub['NEIGHBORHOOD_CLEAN'] = neighbor_obs_sub['NEIGHBORHOOD_CLEAN'].replace({'LA15774-5' : 2,
                                                                                       'LA15237-3' : 3,
                                                                                       'LA15773-7' : 1,
                                                                                       'PMI_Skip' : np.nan,
                                                                                       'LA15236-5' : 0})
neighbor_obs_sub['NEIGHBORHOOD_CLEAN'] = neighbor_obs_sub['NEIGHBORHOOD_CLEAN'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_CLEAN'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### watch out for others

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that in your neighborhood people watch out for each other?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_WATCH'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_WATCH']]
neighbor_obs_sub['NEIGHBORHOOD_WATCH'] = neighbor_obs_sub['NEIGHBORHOOD_WATCH'].replace({'LA15774-5' : 2,
                                                                                       'LA15237-3' : 3,
                                                                                       'LA15773-7' : 1,
                                                                                       'PMI_Skip' : np.nan,
                                                                                       'LA15236-5' : 0})
neighbor_obs_sub['NEIGHBORHOOD_WATCH'] = neighbor_obs_sub['NEIGHBORHOOD_WATCH'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_WATCH'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### housing

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['What is the main type of housing in your neighborhood [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_HOUSING'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_HOUSING']]
neighbor_obs_sub['NEIGHBORHOOD_HOUSING'] = neighbor_obs_sub['NEIGHBORHOOD_HOUSING'].replace({'LA15229-0' : 4,
                                                                                             'LA15231-6' : 2,
                                                                                             'LA30308-3' : 3,
                                                                                             'LA15232-4' : 1,
                                                                                             'LA15233-2' : 0,
                                                                                             'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_HOUSING'] = neighbor_obs_sub['NEIGHBORHOOD_HOUSING'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_HOUSING'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### get along

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['People in this neighborhood generally don\'t get along with each other [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_GET_ALONG'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_GET_ALONG']]
neighbor_obs_sub['NEIGHBORHOOD_GET_ALONG'] = neighbor_obs_sub['NEIGHBORHOOD_GET_ALONG'].replace({'LA15774-5' : 1,
                                                                                                 'LA15237-3' : 0,
                                                                                                 'LA15240-7' : 2,
                                                                                                 'LA15773-7' : 3,
                                                                                                 'PMI_Skip' : np.nan,
                                                                                                 'LA15236-5' : 4 })
neighbor_obs_sub['NEIGHBORHOOD_GET_ALONG'] = neighbor_obs_sub['NEIGHBORHOOD_GET_ALONG'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_GET_ALONG'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### grafitti

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that there is a lot of graffiti in your neighborhood?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_GRAFFITI'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_GRAFFITI']]
neighbor_obs_sub['NEIGHBORHOOD_GRAFFITI'] = neighbor_obs_sub['NEIGHBORHOOD_GRAFFITI'].replace({'LA15236-5' : 3,
                                                                                               'LA15773-7' : 2,
                                                                                               'LA15774-5' : 1,
                                                                                               'LA15237-3' : 0,
                                                                                               'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_GRAFFITI'] = neighbor_obs_sub['NEIGHBORHOOD_GRAFFITI'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_GRAFFITI'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### noise

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that your neighborhood is noisy?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_NOISE'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_NOISE']]
neighbor_obs_sub['NEIGHBORHOOD_NOISE'] = neighbor_obs_sub['NEIGHBORHOOD_NOISE'].replace({'LA15236-5' : 3,
                                                                                         'LA15773-7' : 2,
                                                                                         'LA15774-5' : 1,
                                                                                         'LA15237-3' : 0,
                                                                                         'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_NOISE'] = neighbor_obs_sub['NEIGHBORHOOD_NOISE'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_NOISE'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### unsafe to walk at  night

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['The crime rate in my neighborhood makes it unsafe to go on walks at night [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_UNSAFE_WALK'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_UNSAFE_WALK']]
neighbor_obs_sub['NEIGHBORHOOD_UNSAFE_WALK'] = neighbor_obs_sub['NEIGHBORHOOD_UNSAFE_WALK'].replace({'LA15236-5' : 3,
                                                                                                     'LA15234-0' : 2,
                                                                                                     'LA15235-7' : 1,
                                                                                                     'LA15237-3' : 0,
                                                                                                     'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_UNSAFE_WALK'] = neighbor_obs_sub['NEIGHBORHOOD_UNSAFE_WALK'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_UNSAFE_WALK'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### take good care of homes

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that people in your neighborhood take good care of their houses and apartments?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_CARE'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_CARE']]
neighbor_obs_sub['NEIGHBORHOOD_CARE'] = neighbor_obs_sub['NEIGHBORHOOD_CARE'].replace({'LA15774-5' : 2,
                                                                                       'LA15237-3' : 3,
                                                                                       'LA15773-7' : 1,
                                                                                       'LA15236-5' : 0,
                                                                                       'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_CARE'] = neighbor_obs_sub['NEIGHBORHOOD_CARE'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_CARE'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### a lot of crime

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that there is a lot of crime in your neighborhood?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_ALOT_CRIME'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_ALOT_CRIME']]
neighbor_obs_sub['NEIGHBORHOOD_ALOT_CRIME'] = neighbor_obs_sub['NEIGHBORHOOD_ALOT_CRIME'].replace({'LA15236-5' : 3,
                                                                                                   'LA15773-7' : 2,
                                                                                                   'LA15774-5' : 1,
                                                                                                   'PMI_Skip' : np.nan,
                                                                                                   'LA15237-3' : 0})
neighbor_obs_sub['NEIGHBORHOOD_ALOT_CRIME'] = neighbor_obs_sub['NEIGHBORHOOD_ALOT_CRIME'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_ALOT_CRIME'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### crime rate makes it unsafe to walk during the day

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['The crime rate in my neighborhood makes it unsafe to go on walks during the day. Would you say that you...'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_CRIME_WALK'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_CRIME_WALK']]
neighbor_obs_sub['NEIGHBORHOOD_CRIME_WALK'] = neighbor_obs_sub['NEIGHBORHOOD_CRIME_WALK'].replace({'LA15236-5' : 3,
                                                                                                   'LA15234-0' : 2,
                                                                                                   'PMI_Skip' : np.nan,
                                                                                                   'LA15235-7' : 1,
                                                                                                   'LA15237-3' : 0})
neighbor_obs_sub['NEIGHBORHOOD_CRIME_WALK'] = neighbor_obs_sub['NEIGHBORHOOD_CRIME_WALK'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_CRIME_WALK'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### share same values

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that people in your neighborhood share the same values?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_SAME_VALUES'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_SAME_VALUES']]
neighbor_obs_sub['NEIGHBORHOOD_SAME_VALUES'] = neighbor_obs_sub['NEIGHBORHOOD_SAME_VALUES'].replace({'LA15240-7' : 2,
                                                                                                     'LA15774-5' : 3,
                                                                                                     'LA15237-3' : 4,
                                                                                                     'LA15773-7' : 1,
                                                                                                     'PMI_Skip' : np.nan,
                                                                                                     'LA15236-5' : 0})
neighbor_obs_sub['NEIGHBORHOOD_SAME_VALUES'] = neighbor_obs_sub['NEIGHBORHOOD_SAME_VALUES'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_SAME_VALUES'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### people hanging around

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that there are too many people hanging around on the streets near your home?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_PPL_HANGING_AROUND'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_PPL_HANGING_AROUND']]
neighbor_obs_sub['NEIGHBORHOOD_PPL_HANGING_AROUND'] = neighbor_obs_sub['NEIGHBORHOOD_PPL_HANGING_AROUND'].replace({'LA15236-5' : 3,
                                                                                                     'LA15773-7' : 2,
                                                                                                     'LA15774-5' : 1,
                                                                                                     'PMI_Skip' : np.nan,
                                                                                                     'LA15237-3' : 0})
neighbor_obs_sub['NEIGHBORHOOD_PPL_HANGING_AROUND'] = neighbor_obs_sub['NEIGHBORHOOD_PPL_HANGING_AROUND'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_PPL_HANGING_AROUND'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### trouble with neighbors

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['How much you agree or disagree that you are always having trouble with your neighbors?'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_TROUBLE'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_TROUBLE']]
neighbor_obs_sub['NEIGHBORHOOD_TROUBLE'] = neighbor_obs_sub['NEIGHBORHOOD_TROUBLE'].replace({'LA15236-5' : 3,
                                                                                                     'LA15773-7' : 2,
                                                                                                     'PMI_Skip' : np.nan,
                                                                                                     'LA15774-5' : 1,
                                                                                                     'LA15237-3' : 0})
neighbor_obs_sub['NEIGHBORHOOD_TROUBLE'] = neighbor_obs_sub['NEIGHBORHOOD_TROUBLE'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_TROUBLE'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### stores within walking distance

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['Many shops, stores, markets, or other places to buy things I need are within easy walking distance of my home [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_STORES'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_STORES']]
neighbor_obs_sub['NEIGHBORHOOD_STORES'] = neighbor_obs_sub['NEIGHBORHOOD_STORES'].replace({'LA15236-5' : 0,
                                                                                           'LA15237-3' : 3,
                                                                                           'LA15235-7' : 2,
                                                                                           'LA15234-0' : 1,
                                                                                           'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_STORES'] = neighbor_obs_sub['NEIGHBORHOOD_STORES'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_STORES'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### transit within walking distance

In [ ]:
neighbor_obs_sub = neighbor_obs[neighbor_obs['standard_concept_name'].isin(['It is within a 10-15 minute walk to a transit stop, such as bus, train, trolley, or tram, from my home [PhenX]'])]
print(neighbor_obs_sub['standard_concept_code'].unique())
print(neighbor_obs_sub['standard_vocabulary'].unique())
neighbor_obs_sub ['value_as_string'].value_counts()

In [ ]:
neighbor_obs_sub.rename(columns = {'value_as_string' : 'NEIGHBORHOOD_TRANSIT'}, inplace = True)
neighbor_obs_sub = neighbor_obs_sub[['person_id', 'NEIGHBORHOOD_TRANSIT']]
neighbor_obs_sub['NEIGHBORHOOD_TRANSIT'] = neighbor_obs_sub['NEIGHBORHOOD_TRANSIT'].replace({'LA15237-3' : 3,
                                                                                             'LA15236-5' : 0,
                                                                                             'LA15235-7' : 2,
                                                                                             'LA15234-0' : 1,
                                                                                             'PMI_Skip' : np.nan})
neighbor_obs_sub['NEIGHBORHOOD_TRANSIT'] = neighbor_obs_sub['NEIGHBORHOOD_TRANSIT'].astype('Int64')
print(neighbor_obs_sub['NEIGHBORHOOD_TRANSIT'].unique())
neighbor_obs_sub.head()

In [ ]:
neighbor_obs_clean = neighbor_obs_clean.merge(neighbor_obs_sub, on = 'person_id', how = 'outer')
neighbor_obs_clean.head()

### merge

In [ ]:
print(len(neighbor_obs_clean.index))
print(len(neighbor_obs_clean['person_id'].unique()))

In [ ]:
hf_neighbor_obs = hf_bmi.merge(neighbor_obs_clean, how = 'left', on = 'person_id')
print(len(hf_neighbor_obs.index))
hf_neighbor_obs.head()

## physical activity

### encoding
**Level of physical activity**
- 'LA6751-7' : 1 (moderate)
- 'LA32775-1' : 0 (sedentary)
- 'LA20166-7' : 2 (vigorous exercise intensity)

**To what extent are you able to carry out your everyday physical activities such as walking, climbing stairs, carrying groceries, or moving a chair [PROMIS]**
- 'LA13937-0' : 4 (completely)
- 'LA13938-8 ' : 3 (mostly)
- 'LA13939-6' : 2 (moderately)
- 'LA13940-4' : 1 (a little)
- 'LA6568-5' : 0 (not at all)
- 'PMI_Skip' : np.nan (missing)

**Physical activity**
* skipping, can't understand response codes and only ~100 people have this

**Leisure physical activity**
* skipping, only one response code which is no matching concept and only ~5 people have this

### filter observation df to physical activity

In [ ]:
physical_obs = observation[observation['standard_concept_name'].str.contains('physical|activity|activities', case = False)]
physical_obs['standard_concept_name'].unique()

In [ ]:
physical_obs = physical_obs[~physical_obs['standard_concept_name'].str.contains('Counseling|evaluation|initial|advice', case = False)]
physical_obs['standard_concept_name'].unique()

### everyday physical activity

In [ ]:
physical_obs_sub = physical_obs[physical_obs['standard_concept_name'].isin(['To what extent are you able to carry out your everyday physical activities such as walking, climbing stairs, carrying groceries, or moving a chair [PROMIS]'])]
print(physical_obs_sub['standard_concept_code'].unique())
print(physical_obs_sub['standard_vocabulary'].unique())
physical_obs_sub ['value_as_string'].value_counts()

#### compare values before and after dx

In [ ]:
print(len(physical_obs_sub.index))
pa_plot_df = physical_obs_sub.copy()
hf_date = pheno[['person_id', 'HF_DATE', 'HF']]
pa_plot_df = pa_plot_df.merge(hf_date, on = 'person_id', how = 'inner')
pa_plot_df['DIFF'] = pd.to_datetime(pa_plot_df['HF_DATE'], format = 'ISO8601', utc = True) - pa_plot_df['observation_datetime']
pa_plot_df['DIFF'] = pa_plot_df['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
pa_plot_df['DIFF'] = pa_plot_df['DIFF'].astype(float)

pa_plot_df.sort_values(by = ['person_id', 'DIFF'], inplace = True)
#pa_plot_df.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(pa_plot_df.index))


pa_plot_df = pa_plot_df.rename(columns = {'value_as_string' : 'PA_EVERYDAY'})
pa_plot_df['PA_EVERYDAY'] = pa_plot_df['PA_EVERYDAY'].replace({'LA13937-0' : 4,
                                                                           'LA13938-8' : 3,
                                                                           'LA13939-6' : 2,
                                                                           'LA13940-4' : 1,
                                                                           'LA6568-5' : 0,
                                                                           'PMI_Skip' : np.nan})
pa_plot_before = pa_plot_df[pa_plot_df['DIFF'] >= 0]
pa_plot_after = pa_plot_df[pa_plot_df['DIFF'] < 0]
pa_before_mean = pa_plot_before['PA_EVERYDAY'].mean()
pa_after_mean = pa_plot_after['PA_EVERYDAY'].mean()

sns.kdeplot(pa_plot_after, x = 'PA_EVERYDAY', label = f"Values after HF dx (mean = {pa_after_mean:.2f})")
sns.kdeplot(pa_plot_before, x = 'PA_EVERYDAY', label = f"Values before HF dx (mean = {pa_before_mean:.2f})")
plt.xlabel("Everyday Physical Activity")
plt.ylabel("Density")
plt.title("AOU Everyday Physical Activity Values Before and After HF dx")
plt.legend()
plt.savefig("AOU.PA_everyday.before_after_HF_dx.png", dpi = 300)
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (18, 18))
axes = axes.flatten()

# BMI
sns.kdeplot(bmi_plot_after, x = 'value_as_number', label = f"Values after HF dx (mean = {bmi_after_mean:.2f})", ax = axes[0])
sns.kdeplot(bmi_plot_before, x = 'value_as_number', label = f"Values before HF dx (mean = {bmi_before_mean:.2f})", ax = axes[0])
axes[0].set_title("AOU BMI Values Before and After HF dx")
axes[0].set_xlabel("BMI")
axes[0].set_ylabel("Density")
axes[0].legend(loc = 'upper right')

# DBP
sns.kdeplot(diastolic_plot_after, x = 'value_as_number', label = f"Values after HF dx (mean = {diastolic_after_mean:.2f})", ax = axes[1])
sns.kdeplot(diastolic_plot_before, x = 'value_as_number', label = f"Values before HF dx (mean = {diastolic_before_mean:.2f})", ax = axes[1])
axes[1].set_title("AOU Diastolic Blood Pressure Values Before and After HF dx")
axes[1].set_xlabel("Diastolic Blood Pressure")
axes[1].set_ylabel("Density")
axes[1].legend()

# PA
sns.kdeplot(pa_plot_after, x = 'PA_EVERYDAY', label = f"Values after HF dx (mean = {pa_after_mean:.2f})", ax = axes[2])
sns.kdeplot(pa_plot_before, x = 'PA_EVERYDAY', label = f"Values before HF dx (mean = {pa_before_mean:.2f})", ax = axes[2])
axes[2].set_title("AOU Everyday Physical Activity Values Before and After HF dx")
axes[2].set_xlabel("Everyday Physical Activity")
axes[2].set_ylabel("Density")
axes[2].legend(loc = 'upper right')

# Glucose
sns.kdeplot(glucose_plot_after, x = 'value_as_number', label = f"Values after HF dx (mean = {glucose_after_mean:.2f})", ax = axes[3])
sns.kdeplot(glucose_plot_before, x = 'value_as_number', label = f"Values before HF dx (mean = {glucose_before_mean:.2f})", ax = axes[3])
axes[3].set_title("AOU Glucose Values Before and After HF dx")
axes[3].set_xlabel("Glucose")
axes[3].set_ylabel("Density")
axes[3].legend()

# HbA1c
sns.kdeplot(hba1c_plot_after, x = 'value_as_number', label = f"Values after HF dx (mean = {hba1c_after_mean:.2f})", ax = axes[4])
sns.kdeplot(hba1c_plot_before, x = 'value_as_number', label = f"Values before HF dx (mean = {hba1c_before_mean:.2f})", ax = axes[4])
axes[4].set_title("AOU HbA1c Values Before and After HF dx")
axes[4].set_xlabel("HbA1c")
axes[4].set_ylabel("Density")
axes[4].legend()

# HDL
sns.kdeplot(hdl_plot_after, x = 'value_as_number', label = f"Values after HF dx (mean = {hdl_after_mean:.2f})", ax = axes[5])
sns.kdeplot(hdl_plot_before, x = 'value_as_number', label = f"Values before HF dx (mean = {hdl_before_mean:.2f})", ax = axes[5])
axes[5].set_title("AOU HDL Values Before and After HF dx")
axes[5].set_xlabel("HDL")
axes[5].set_ylabel("Density")
axes[5].legend(loc = 'upper right')

# LDL
sns.kdeplot(ldl_plot_after, x = 'value_as_number', label = f"Values after HF dx (mean = {ldl_after_mean:.2f})", ax = axes[6])
sns.kdeplot(ldl_plot_before, x = 'value_as_number', label = f"Values before HF dx (mean = {ldl_before_mean:.2f})", ax = axes[6])
axes[6].set_title("AOU LDL Values Before and After HF dx")
axes[6].set_xlabel("LDL")
axes[6].set_ylabel("Density")
axes[6].legend()

# SBP
sns.kdeplot(systolic_plot_after, x = 'value_as_number', label = f"Values after HF dx (mean = {systolic_after_mean:.2f})", ax = axes[7])
sns.kdeplot(systolic_plot_before, x = 'value_as_number', label = f"Values before HF dx (mean = {systolic_before_mean:.2f})", ax = axes[7])
axes[7].set_title("AOU Systolic Blood Pressure Values Before and After HF dx")
axes[7].set_xlabel("Systolic Blood Pressure")
axes[7].set_ylabel("Density")
axes[7].legend()

# Trig
sns.kdeplot(trig_plot_after, x = 'value_as_number', label = f"Values after HF dx (mean = {trig_after_mean:.2f})", ax = axes[8])
sns.kdeplot(trig_plot_before, x = 'value_as_number', label = f"Values before HF dx (mean = {trig_before_mean:.2f})", ax = axes[8])
axes[8].set_title("AOU Triglyceride Values Before and After HF dx")
axes[8].set_xlabel("Triglyceride")
axes[8].set_ylabel("Density")
axes[8].legend()

plt.tight_layout()
plt.savefig("AOU.all_variables.before_after_HF_dx.png", dpi = 300)
plt.show()

#### finish cleaning

In [ ]:
print(len(physical_obs_sub.index))
physical_obs_sub = physical_obs_sub.merge(hf_date, on = 'person_id', how = 'inner')
physical_obs_sub['DIFF'] = pd.to_datetime(physical_obs_sub['HF_DATE'], utc = True) - physical_obs_sub['observation_datetime']
physical_obs_sub['DIFF'] = physical_obs_sub['DIFF'].astype(str).str.replace(r' days.*','', regex = True)
physical_obs_sub['DIFF'] = physical_obs_sub['DIFF'].astype(float)

physical_obs_sub = physical_obs_sub[physical_obs_sub['DIFF'] >= 0]
physical_obs_sub.sort_values(by = ['person_id', 'DIFF'], inplace = True)
physical_obs_sub.drop_duplicates(subset = ['person_id'], keep = 'first', inplace = True)
print(len(physical_obs_sub.index))

In [ ]:
physical_obs_sub.rename(columns = {'value_as_string' : 'PA_EVERYDAY'}, inplace = True)
physical_obs_sub = physical_obs_sub[['person_id', 'PA_EVERYDAY']]
physical_obs_sub['PA_EVERYDAY'] = physical_obs_sub['PA_EVERYDAY'].replace({'LA13937-0' : 4,
                                                                           'LA13938-8' : 3,
                                                                           'LA13939-6' : 2,
                                                                           'LA13940-4' : 1,
                                                                           'LA6568-5' : 0,
                                                                           'PMI_Skip' : np.nan})
physical_obs_sub['PA_EVERYDAY'] = physical_obs_sub['PA_EVERYDAY'].astype('Int64')
print(physical_obs_sub['PA_EVERYDAY'].unique())
physical_obs_sub.head()

### merge

In [ ]:
print(len(physical_obs_sub['person_id'].unique()))
print(len(physical_obs_sub.index))

In [ ]:
hf_physical_obs = hf_neighbor_obs.merge(physical_obs_sub, on = 'person_id', how = 'left')
print(len(hf_physical_obs.index))
hf_physical_obs.head()

## nutrition

### encoding
**Noncompliance with dietary regimen**
* skipping bc this is an icd code

**Nutritional status**
* skipping bc snomed code means bmi and only 15 people

**Nutritional observable**
* skipping bc only 5 people have this

**Nutritional finding**
* skipping bc only 22 people have this

**Finding of nutritional status**
* skipping bc all missing

### filter observation df to nutrition

In [ ]:
nutrition_obs = observation[observation['standard_concept_name'].str.contains('dietary|dietitian|nutrition|nutritional|diet', case = False)]
nutrition_obs['standard_concept_name'].unique()

In [ ]:
nutrition_obs = nutrition_obs[~nutrition_obs['standard_concept_name'].str.contains('disorders|deficiency|monitoring|referral|assessment|counseling|alteration|impaired|referral|dosages', case = False)]
nutrition_obs['standard_concept_name'].unique()

## smoking

### encoding
- 0 = non-smoker
- 1 = former smoker
- 2 = current smoker

**Cigarette smoking tobacco**
- skipped, because all no matching concept

**How many cigarettes do you smoke per day now [PhenX]**
- value as number > 0 : 2 (smoker)
- value as number = 0 : 0 (nonsmoker or former)

**Cigar Smoking: Current Cigar Frequency**
- 'LA6568-5' : 0 (Not at all)
- 'LA15776-0' : 2 (Some days)
- 'LA14799-3' : 2 (Every day)
- 'PMI_Skip' : np.nan (missing)

**Do you now smoke cigarettes every day, some days, or not at all [PhenX]**
- 'LA6568-5' : 0 (Not at all)
- 'LA14799-3' : 2 (Every day)
- 'LA15776-0' : 2 (Some days)
- 'PMI_Skip' : np.nan (missing)

**Cigar Smoking: Cigar Smoke Participant**
- 'LA32-8' : 0 (No)
- 'LA33-6' : 2 (Yes)
- 'PMI_Skip' : np.nan (missing)

**Tobacco smoking status**
- [r'^\d.*','No matching concept',r'OMOP.*'] : np.nan (missing)
- 'LA18978-9' : 0 (Never smoker)
- 'LA18979-7' : np.nan (Smoker, current status unknown)
- 'LA18980-5' : np.nan (Unknown if ever smoked)
- 'LA15920-4' : 1 (Former smoker)
- 'LA18976-3' : 2 (Current every day smoker)
- 'LA18977-1' : 2 (Current some day smoker)
- 'LA18982-1' : np.nan (Light tobacco smoker)
- 'LA18981-3' : np.nan (Heavy tobacco smoker)

**Smoked at least 100 cigarettes in entire life**
- 'LA32-8' : 0 (No)
- 'LA33-6' : 2 (Yes)
- 'PMI_Skip' : np.nan (missing)

**Pipe smoking tobacco**
- skipped due to no values

**Cigar smoking tobacco**
- skipped due to no values

**Attempt Quit Smoking: Completely Quit Age**
- value as number < AGE = 1 (former smoker)
- value as number > AGE = 2 (current smoker)

**Date quit tobacco smoking**
- skipping- only 500 values and numbers make no sense

### filter observation df to smoking

In [ ]:
smoking_obs = observation[observation['standard_concept_name'].str.contains('smoking|smoke|smoked', case = False)]
smoking_obs['standard_concept_name'].unique()

In [ ]:
smoking_obs = smoking_obs[~smoking_obs['standard_concept_name'].str.contains('wake|serious|old|verbal|Hookah|Electronic|counseling|months|anesthesia|products|second|Family|duration', case = False)]
smoking_obs['standard_concept_name'].unique()

### cigs per day now

In [ ]:
smoking_obs_sub = smoking_obs[smoking_obs['standard_concept_name'].isin(['How many cigarettes do you smoke per day now [PhenX]'])]
print(smoking_obs_sub['standard_concept_code'].unique())
print(smoking_obs_sub['standard_vocabulary'].unique())
smoking_obs_sub ['value_as_number'].describe()

In [ ]:
smoking_obs_sub.rename(columns = {'value_as_number' : 'SMOKING_CIGS_DAY'}, inplace = True)
smoking_obs_sub = smoking_obs_sub[['person_id', 'SMOKING_CIGS_DAY']]
smoking_obs_sub['SMOKING_CIGS_DAY'] = np.where(smoking_obs_sub['SMOKING_CIGS_DAY'] > 0, 2, 0)
smoking_obs_sub['SMOKING_CIGS_DAY'] = smoking_obs_sub['SMOKING_CIGS_DAY'].astype('Int64')
print(smoking_obs_sub['SMOKING_CIGS_DAY'].value_counts())
print(len(smoking_obs_sub.index))
print(len(smoking_obs_sub['person_id'].unique()))
smoking_obs_sub.head()

In [ ]:
smoking_obs_clean = smoking_obs_sub

### cigar smoking frequency

In [ ]:
smoking_obs_sub = smoking_obs[smoking_obs['standard_concept_name'].isin(['Cigar Smoking: Current Cigar Frequency'])]
print(smoking_obs_sub['standard_concept_code'].unique())
print(smoking_obs_sub['standard_vocabulary'].unique())
smoking_obs_sub ['value_as_string'].value_counts()

In [ ]:
smoking_obs_sub.rename(columns = {'value_as_string' : 'SMOKING_CIGAR_FREQ'}, inplace = True)
smoking_obs_sub = smoking_obs_sub[['person_id', 'SMOKING_CIGAR_FREQ']]
smoking_obs_sub['SMOKING_CIGAR_FREQ'] = smoking_obs_sub['SMOKING_CIGAR_FREQ'].replace({'LA6568-5' : 0,
                                                                                       'LA15776-0' : 2,
                                                                                       'LA14799-3' : 2,
                                                                                       'PMI_Skip' : np.nan})
smoking_obs_sub['SMOKING_CIGAR_FREQ'] = smoking_obs_sub['SMOKING_CIGAR_FREQ'].astype('Int64')
print(smoking_obs_sub['SMOKING_CIGAR_FREQ'].value_counts())
print(len(smoking_obs_sub.index))
print(len(smoking_obs_sub['person_id'].unique()))
smoking_obs_sub.head()

In [ ]:
smoking_obs_clean = smoking_obs_clean.merge(smoking_obs_sub, on = 'person_id', how = 'outer')
smoking_obs_clean.head()

### cig freq

In [ ]:
smoking_obs_sub = smoking_obs[smoking_obs['standard_concept_name'].isin(['Do you now smoke cigarettes every day, some days, or not at all [PhenX]'])]
print(smoking_obs_sub['standard_concept_code'].unique())
print(smoking_obs_sub['standard_vocabulary'].unique())
smoking_obs_sub ['value_as_string'].value_counts()

In [ ]:
smoking_obs_sub.rename(columns = {'value_as_string' : 'SMOKING_CIG_FREQ'}, inplace = True)
smoking_obs_sub = smoking_obs_sub[['person_id', 'SMOKING_CIG_FREQ']]
smoking_obs_sub['SMOKING_CIG_FREQ'] = smoking_obs_sub['SMOKING_CIG_FREQ'].replace({'LA6568-5' : 0,
                                                                                   'LA15776-0' : 2,
                                                                                   'LA14799-3' : 2,
                                                                                   'PMI_Skip' : np.nan})
smoking_obs_sub['SMOKING_CIG_FREQ'] = smoking_obs_sub['SMOKING_CIG_FREQ'].astype('Int64')
print(smoking_obs_sub['SMOKING_CIG_FREQ'].value_counts())
print(len(smoking_obs_sub.index))
print(len(smoking_obs_sub['person_id'].unique()))
smoking_obs_sub.head()

In [ ]:
smoking_obs_clean = smoking_obs_clean.merge(smoking_obs_sub, on = 'person_id', how = 'outer')
smoking_obs_clean.head()

### cigar smoking participant

In [ ]:
smoking_obs_sub = smoking_obs[smoking_obs['standard_concept_name'].isin(['Cigar Smoking: Cigar Smoke Participant'])]
print(smoking_obs_sub['standard_concept_code'].unique())
print(smoking_obs_sub['standard_vocabulary'].unique())
smoking_obs_sub ['value_as_string'].value_counts()

In [ ]:
smoking_obs_sub.rename(columns = {'value_as_string' : 'SMOKING_CIGAR'}, inplace = True)
smoking_obs_sub = smoking_obs_sub[['person_id', 'SMOKING_CIGAR']]
smoking_obs_sub['SMOKING_CIGAR'] = smoking_obs_sub['SMOKING_CIGAR'].replace({'LA32-8' : 0,
                                                                             'LA33-6' : 2,
                                                                             'PMI_Skip' : np.nan})
smoking_obs_sub['SMOKING_CIGAR'] = smoking_obs_sub['SMOKING_CIGAR'].astype('Int64')
print(smoking_obs_sub['SMOKING_CIGAR'].value_counts())
print(len(smoking_obs_sub.index))
print(len(smoking_obs_sub['person_id'].unique()))
smoking_obs_sub.head()

In [ ]:
smoking_obs_clean = smoking_obs_clean.merge(smoking_obs_sub, on = 'person_id', how = 'outer')
smoking_obs_clean.head()

### tobacco smoking status

In [ ]:
smoking_obs_sub = smoking_obs[smoking_obs['standard_concept_name'].isin(['Tobacco smoking status'])]
print(smoking_obs_sub['standard_concept_code'].unique())
print(smoking_obs_sub['standard_vocabulary'].unique())
smoking_obs_sub ['value_as_string'].value_counts()

In [ ]:
smoking_obs_sub.rename(columns = {'value_as_string' : 'SMOKING_TOBACCO'}, inplace = True)
smoking_obs_sub = smoking_obs_sub[['person_id', 'SMOKING_TOBACCO', 'observation_datetime']]
smoking_obs_sub['observation_datetime'] = pd.to_datetime(smoking_obs_sub['observation_datetime'])
smoking_obs_sub['SMOKING_TOBACCO'] = smoking_obs_sub['SMOKING_TOBACCO'].replace([r'^\d.*','No matching concept',r'OMOP.*'], np.nan, regex = True)
smoking_obs_sub['SMOKING_TOBACCO'] = smoking_obs_sub['SMOKING_TOBACCO'].replace({'LA18978-9' : 0,
                                                                                 'LA18979-7' : np.nan,
                                                                                 'LA18980-5' : np.nan,
                                                                                 'LA15920-4' : 1,
                                                                                 'LA18976-3' : 2,
                                                                                 'LA18977-1' : 2,
                                                                                 'LA18982-1' : np.nan,
                                                                                 'LA18981-3' : np.nan})
print(len(smoking_obs_sub.index))
print(len(smoking_obs_sub['person_id'].unique()))

In [ ]:
no_na = smoking_obs_sub.dropna(subset = ['SMOKING_TOBACCO'])

In [ ]:
test1 = smoking_obs_sub[smoking_obs_sub['person_id'].duplicated(keep = 'last')].sort_values(by = 'person_id')
print(len(test1.index))

In [ ]:
test2 = smoking_obs_sub[smoking_obs_sub[['person_id', 'SMOKING_TOBACCO']].duplicated(keep = 'last')].sort_values(by = 'person_id')
print(len(test2.index))

In [ ]:
test3 = test1[~test1['person_id'].isin(test2['person_id'])].sort_values(by = 'person_id')
print(len(test3.index))

In [ ]:
test4 = smoking_obs_sub[smoking_obs_sub['person_id'].isin(test3['person_id'])].sort_values(by = 'person_id')
print(len(test4.index))

In [ ]:
hf_pgs_no_missing = pd.read_csv('HF_Clinical_PGS.phenotype.no_missing.variable_transformation.csv')

In [ ]:
test5 = hf_pgs_no_missing[hf_pgs_no_missing['person_id'].isin(test3['person_id'])]
print(len(test5.index))

In [ ]:
smoking_obs_sub[smoking_obs_sub[['person_id', 'SMOKING_TOBACCO']].duplicated(keep = 'last')].sort_values(by = 'person_id')['SMOKING_TOBACCO'].value_counts(dropna = False)

In [ ]:
smoking_obs_sub.rename(columns = {'value_as_string' : 'SMOKING_TOBACCO'}, inplace = True)
smoking_obs_sub = smoking_obs_sub[['person_id', 'SMOKING_TOBACCO', 'observation_datetime']]
smoking_obs_sub['observation_datetime'] = pd.to_datetime(smoking_obs_sub['observation_datetime'])
smoking_obs_sub['SMOKING_TOBACCO'] = smoking_obs_sub['SMOKING_TOBACCO'].replace([r'^\d.*','No matching concept',r'OMOP.*'], np.nan, regex = True)
smoking_obs_sub['SMOKING_TOBACCO'] = smoking_obs_sub['SMOKING_TOBACCO'].replace({'LA18978-9' : 0,
                                                                                 'LA18979-7' : np.nan,
                                                                                 'LA18980-5' : np.nan,
                                                                                 'LA15920-4' : 1,
                                                                                 'LA18976-3' : 2,
                                                                                 'LA18977-1' : 2,
                                                                                 'LA18982-1' : np.nan,
                                                                                 'LA18981-3' : np.nan})
smoking_obs_sub.sort_values(by = ['person_id', 'observation_datetime'], inplace = True, ascending = False)
print(len(smoking_obs_sub.index))
smoking_obs_sub.drop_duplicates(subset = 'person_id', keep = 'first', inplace = True)
print(len(smoking_obs_sub.index))
smoking_obs_sub.drop(columns = ['observation_datetime'], inplace = True)
smoking_obs_sub['SMOKING_TOBACCO'] = smoking_obs_sub['SMOKING_TOBACCO'].astype('Int64')
print(smoking_obs_sub['SMOKING_TOBACCO'].value_counts(dropna = False))
print(len(smoking_obs_sub.index))
print(len(smoking_obs_sub['person_id'].unique()))
smoking_obs_sub.head()

In [ ]:
smoking_obs_clean = smoking_obs_clean.merge(smoking_obs_sub, on = 'person_id', how = 'outer')
smoking_obs_clean.head()

### at least 100 cigs in life

In [ ]:
smoking_obs_sub = smoking_obs[smoking_obs['standard_concept_name'].isin(['Smoked at least 100 cigarettes in entire life'])]
print(smoking_obs_sub['standard_concept_code'].unique())
print(smoking_obs_sub['standard_vocabulary'].unique())
smoking_obs_sub ['value_as_string'].value_counts()

In [ ]:
smoking_obs_sub.rename(columns = {'value_as_string' : 'SMOKING_100_CIGS'}, inplace = True)
smoking_obs_sub = smoking_obs_sub[['person_id', 'SMOKING_100_CIGS']]
smoking_obs_sub['SMOKING_100_CIGS'] = smoking_obs_sub['SMOKING_100_CIGS'].replace({'LA32-8' : 0,
                                                                                   'LA33-6' : 2,
                                                                                   'PMI_Skip' : np.nan})
smoking_obs_sub['SMOKING_100_CIGS'] = smoking_obs_sub['SMOKING_100_CIGS'].astype('Int64')
print(smoking_obs_sub['SMOKING_100_CIGS'].value_counts())
print(len(smoking_obs_sub.index))
print(len(smoking_obs_sub['person_id'].unique()))
smoking_obs_sub.head()

In [ ]:
smoking_obs_clean = smoking_obs_clean.merge(smoking_obs_sub, on = 'person_id', how = 'outer')
smoking_obs_clean.head()

### completely quit age

In [ ]:
smoking_obs_sub = smoking_obs[smoking_obs['standard_concept_name'].isin(['Attempt Quit Smoking: Completely Quit Age'])]
print(smoking_obs_sub['standard_concept_code'].unique())
print(smoking_obs_sub['standard_vocabulary'].unique())
smoking_obs_sub ['value_as_number'].describe().apply(lambda x: f'{x:,.2f}')

In [ ]:
print(len(smoking_obs_sub.index))
print(len(smoking_obs_sub['person_id'].unique()))

In [ ]:
age_df = hf_case_control[['person_id', 'AGE']]
print(len(age_df.index))
smoking_obs_age = age_df.merge(smoking_obs_sub, on = 'person_id', how = 'inner')
print(len(smoking_obs_age.index))

In [ ]:
smoking_obs_age = smoking_obs_age[['person_id', 'value_as_number', 'AGE']]
smoking_obs_age['SMOKING_QUIT_AGE'] = np.where(smoking_obs_age['value_as_number'] < smoking_obs_age['AGE'], 1, 2)
smoking_obs_age['SMOKING_QUIT_AGE'] = smoking_obs_age['SMOKING_QUIT_AGE'].astype('Int64')
smoking_obs_age.drop(columns = ['AGE', 'value_as_number'], inplace = True)
print(smoking_obs_age['SMOKING_QUIT_AGE'].value_counts())
print(len(smoking_obs_age.index))
print(len(smoking_obs_age['person_id'].unique()))
smoking_obs_age.head()

In [ ]:
smoking_obs_clean = smoking_obs_clean.merge(smoking_obs_age, on = 'person_id', how = 'outer')
smoking_obs_clean.head()

### date quit smoking

In [ ]:
smoking_obs_sub = smoking_obs[smoking_obs['standard_concept_name'].isin(['Date quit tobacco smoking'])]
print(smoking_obs_sub['standard_concept_code'].unique())
print(smoking_obs_sub['standard_vocabulary'].unique())
print(len(smoking_obs_sub.index))
print(len(smoking_obs_sub['person_id'].unique()))
print(smoking_obs_sub ['value_as_number'].describe().apply(lambda x: f'{x:,.2f}'))
smoking_obs_sub ['value_as_string'].value_counts()

### create combined column and subset

In [ ]:
former_smoker = smoking_obs_clean[(smoking_obs_clean['SMOKING_TOBACCO'] == 1) | (smoking_obs_clean['SMOKING_QUIT_AGE'] == 1)]
print(len(former_smoker.index))
former_smoker[['SMOKING_TOBACCO', 'SMOKING_QUIT_AGE']].value_counts(dropna = False)

In [ ]:
former_smoker = former_smoker[((former_smoker['SMOKING_TOBACCO'].isna() == True) | (former_smoker['SMOKING_TOBACCO'] == 1)) & ((former_smoker['SMOKING_QUIT_AGE'].isna() == True) | (former_smoker['SMOKING_QUIT_AGE'] == 1))]
print(len(former_smoker.index))
print(former_smoker[['SMOKING_TOBACCO', 'SMOKING_QUIT_AGE']].value_counts(dropna = False))
former_smoker['SMOKING'] = 1
former_smoker.head()

In [ ]:
smoking_cols = [col for col in smoking_obs_clean.columns if col.startswith('SMOKING_')]
print(smoking_cols)
no_former_smoker = smoking_obs_clean[~smoking_obs_clean['person_id'].isin(former_smoker['person_id'])]
print(len(no_former_smoker.index))
no_former_smoker['SMOKING'] = no_former_smoker[smoking_cols].sum(axis=1)
print(no_former_smoker['SMOKING'].value_counts(dropna = False))
no_former_smoker.head()

In [ ]:
no_former_smoker['SMOKING'] = np.where(no_former_smoker['SMOKING']> 0, 2, 0)
no_former_smoker['SMOKING'] = no_former_smoker['SMOKING'].astype('Int64')
print(no_former_smoker['SMOKING'].value_counts(dropna = False))

In [ ]:
smoking_obs_clean_final = pd.concat([no_former_smoker, former_smoker], axis = 0)
print(len(smoking_obs_clean_final.index))
smoking_obs_clean_final['SMOKING'].value_counts(dropna = False)

In [ ]:
smoking_obs_clean_sub = smoking_obs_clean_final[['person_id', 'SMOKING']]
smoking_obs_clean_sub.head()

### merge

In [ ]:
print(len(smoking_obs_clean_sub['person_id'].unique()))
print(len(smoking_obs_clean_sub.index))

In [ ]:
hf_smoking_obs = hf_physical_obs.merge(smoking_obs_clean_sub, on = 'person_id', how = 'left')
print(len(hf_smoking_obs.index))
print(hf_smoking_obs['SMOKING'].value_counts(dropna = False))
hf_smoking_obs.head()

## income

### encoding

**Financially poor**
- skipping due to no real value and low count

**Total combined household income range in last year**
- 'AnnualIncome_less10k' : 0
- 'AnnualIncome_10k25k' : 1
- 'AnnualIncome_25k35k' : 2
- 'AnnualIncome_35k50k' : 3
- 'AnnualIncome_50k75k' : 4
- 'AnnualIncome_75k100k' : 5
- 'AnnualIncome_100k150k' : 6
- 'AnnualIncome_150k200k' : 7
- 'AnnualIncome_more200k' : 8
- 'PMI_Skip' : np.nan

**Low income**
- skipping due to no real value and low count

### filter observation df to income

In [ ]:
income_obs = observation[observation['standard_concept_name'].str.contains('income|Financially|financial', case = False)]
income_obs['standard_concept_name'].unique()

### total combined household income

In [ ]:
income_obs_sub = income_obs[income_obs['standard_concept_name'].isin(['Total combined household income range in last year'])]
print(income_obs_sub['standard_concept_code'].unique())
print(income_obs_sub['standard_vocabulary'].unique())
income_obs_sub ['value_as_string'].value_counts()

In [ ]:
income_obs_sub.rename(columns = {'value_as_string' : 'INCOME_ANNUAL_HOUSEHOLD'}, inplace = True)
income_obs_sub = income_obs_sub[['person_id', 'INCOME_ANNUAL_HOUSEHOLD']]
income_obs_sub['INCOME_ANNUAL_HOUSEHOLD'] = income_obs_sub['INCOME_ANNUAL_HOUSEHOLD'].replace({'AnnualIncome_less10k' : 0,
                                                                                               'AnnualIncome_10k25k' : 1,
                                                                                               'AnnualIncome_25k35k' : 2,
                                                                                               'AnnualIncome_35k50k' : 3,
                                                                                               'AnnualIncome_50k75k' : 4,
                                                                                               'AnnualIncome_75k100k' : 5,
                                                                                               'AnnualIncome_100k150k' : 6,
                                                                                               'AnnualIncome_150k200k' : 7,
                                                                                               'AnnualIncome_more200k' : 8,
                                                                                               'PMI_Skip' : np.nan})
income_obs_sub['INCOME_ANNUAL_HOUSEHOLD'] = income_obs_sub['INCOME_ANNUAL_HOUSEHOLD'].astype('Int64')
print(income_obs_sub['INCOME_ANNUAL_HOUSEHOLD'].value_counts())
print(len(income_obs_sub.index))
print(len(income_obs_sub['person_id'].unique()))
income_obs_sub.head()

In [ ]:
income_obs_clean = income_obs_sub

### merge

In [ ]:
income_obs_clean.rename(columns = {'INCOME_ANNUAL_HOUSEHOLD' : 'INCOME'}, inplace = True)
income_obs_clean.head()

In [ ]:
print(len(income_obs_clean['person_id'].unique()))
print(len(income_obs_clean.index))

In [ ]:
hf_income_obs = hf_smoking_obs.merge(income_obs_clean, on = 'person_id', how = 'left')
print(len(hf_income_obs.index))
print(hf_income_obs['INCOME'].value_counts(dropna = False))
hf_income_obs.head()

## education

### encoding
**What is the highest grade or level of schooling you completed [SAMHSA]**
- skipping bc values make no sense

**Education and/or schooling finding**
- skipping bc values make no sense

**Lack of education**
- skipping bc no values

**Highest level of education**
- 'LA15609-3' : 3 (3rd grade)
- 'LA15611-9' : 5 (5th grade)
- 'LA15612-7' : 6 (6th grade)
- 'LA15613-5' : 7 (7th grade)
- 'LA15614-3' : 8 (8th grade)
- 'LA15615-0' : 9 (9th grade)
- 'LA15616-8' : 10 (10th grade)
- 'LA15617-6' : 11 (11th grade)
- 'LA15618-4' : 12 (12th grade, no diploma)
- 'LA15564-0' : 13 (High school graduate)
- 'LA15619-2' : 13 (GED or equivalent)
- 'LA12457-0' : 13 (High school graduate or GED completed)
- 'LA15620-0' : 14 (Some college, no degree)
- 'LA15622-6' : 15 (Associate degree, academic program)
- 'LA15621-8' : 15 (Associate degree, occupational, technical, or vocational program)
- 'LA40-1' : np.nan (Some college)
- 'LA12460-4' : 16 (Bachelor's degree (e.g., BA, AB, BS))
- 'LA42-7' : 17 np.nan
- 'LA12461-2' : 17 (Master's degree (e.g., MA, MS, MEng, MEd, MSW, MBA))
- 'LA12462-0' : 18 (Doctorate)
- 'LA15625-9' : 18 (Professional school degree)
- 'No matching concept' : np.nan (missing)

**Years of education [#] - Reported**
- number of years

**Staff education**
- skipped

**Received doctorate education**
- skipped


### filter observation df to education

In [ ]:
education_obs = observation[observation['standard_concept_name'].str.contains('education|grade', case = False)]
education_obs['standard_concept_name'].unique()

In [ ]:
education_obs = education_obs[~education_obs['standard_concept_name'].str.contains('problem|program|explicit|screenings|waiver|iep|Health', case = False)]
education_obs['standard_concept_name'].unique()

### highest level of education

In [ ]:
education_obs_sub = education_obs[education_obs['standard_concept_name'].isin(['Highest level of education'])]
print(education_obs_sub['standard_concept_code'].unique())
print(education_obs_sub['standard_vocabulary'].unique())
education_obs_sub ['value_as_string'].value_counts()

In [ ]:
education_obs_sub.rename(columns = {'value_as_string' : 'EDUCATION_HIGHEST'}, inplace = True)
education_obs_sub = education_obs_sub[['person_id', 'EDUCATION_HIGHEST', 'observation_datetime']]
education_obs_sub['observation_datetime'] = pd.to_datetime(education_obs_sub['observation_datetime'])
education_obs_sub.sort_values(by = ['person_id', 'observation_datetime'], inplace = True, ascending = False)
print(len(education_obs_sub.index))
education_obs_sub.drop_duplicates(subset = 'person_id', keep = 'first', inplace = True)
print(len(education_obs_sub.index))
education_obs_sub.drop(columns = ['observation_datetime'], inplace = True)
education_obs_sub['EDUCATION_HIGHEST'] = education_obs_sub['EDUCATION_HIGHEST'].replace({'LA15609-3' : 3,
                                                                                         'LA15611-9' : 5,
                                                                                         'LA15612-7' : 6,
                                                                                         'LA15613-5' : 7,
                                                                                         'LA15614-3' : 8,
                                                                                         'LA15615-0' : 9,
                                                                                         'LA15616-8' : 10,
                                                                                         'LA15617-6' : 11,
                                                                                         'LA15618-4' : 12,
                                                                                         'LA15564-0' : 13,
                                                                                         'LA15619-2' : 13,
                                                                                         'LA12457-0' : 13,
                                                                                         'LA15620-0' : 14,
                                                                                         'LA15622-6' : 15,
                                                                                         'LA15621-8' : 15,
                                                                                         'LA40-1' : np.nan,
                                                                                         'LA12460-4' : 16,
                                                                                         'LA42-7' : np.nan,
                                                                                         'LA12461-2' : 17,
                                                                                         'LA12462-0' : 18,
                                                                                         'LA15625-9' : 18,
                                                                                         'No matching concept' : np.nan})
education_obs_sub['EDUCATION_HIGHEST'] = education_obs_sub['EDUCATION_HIGHEST'].astype('Int64')
print(education_obs_sub['EDUCATION_HIGHEST'].value_counts())
print(len(education_obs_sub.index))
print(len(education_obs_sub['person_id'].unique()))
education_obs_sub.head()

In [ ]:
education_obs_clean = education_obs_sub

### years of education

In [ ]:
education_obs_sub = education_obs[education_obs['standard_concept_name'].isin(['Years of education [#] - Reported'])]
print(education_obs_sub['standard_concept_code'].unique())
print(education_obs_sub['standard_vocabulary'].unique())
education_obs_sub['value_as_number'].value_counts()

In [ ]:
education_obs_sub.rename(columns = {'value_as_number' : 'EDUCATION_YEARS'}, inplace = True)
education_obs_sub = education_obs_sub[['person_id', 'EDUCATION_YEARS', 'observation_datetime']]
education_obs_sub['observation_datetime'] = pd.to_datetime(education_obs_sub['observation_datetime'])
education_obs_sub.sort_values(by = ['person_id', 'observation_datetime'], inplace = True, ascending = False)
print(len(education_obs_sub.index))
education_obs_sub.drop_duplicates(subset = 'person_id', keep = 'first', inplace = True)
print(len(education_obs_sub.index))
education_obs_sub.drop(columns = ['observation_datetime'], inplace = True)
education_obs_sub['EDUCATION_YEARS'] = education_obs_sub['EDUCATION_YEARS'].astype('Int64')
print(education_obs_sub['EDUCATION_YEARS'].value_counts())
print(len(education_obs_sub.index))
print(len(education_obs_sub['person_id'].unique()))
education_obs_sub.head()

In [ ]:
education_obs_clean = education_obs_clean.merge(education_obs_sub, on = 'person_id', how = 'outer')
education_obs_clean.head()

### merge

In [ ]:
print(len(education_obs_clean['person_id'].unique()))
print(len(education_obs_clean.index))

In [ ]:
hf_education_obs = hf_income_obs.merge(education_obs_clean, on = 'person_id', how = 'left')
print(len(hf_education_obs.index))
print(hf_education_obs['INCOME'].value_counts(dropna = False))
hf_education_obs.head()

## single living status

### encoding
**Marital status**
- skipping bc codes make no sense

**Marital status [NHANES]**
- 'LA48-4' : 0 (Married)
- 'LA17717-2' : 1 (Single)
- 'LA47-6' : 1 (Never married)
- 'LA51-8' : 1 (Divorced)
- 'LA49-2' : 1 (Widowed)
- 'LA15605-1' : 0 (Living with partner)
- 'No matching concept': np.nan (missing)
- 'LA10058-8': np.nan (Patient refused)
- 'LA4288-2' : 1 (Separated)
- 'LA4489-6': np.nan (unknown)
- 'LA30381-0' : 0 (Significant other)
- 'LA46-8' : np.nan (Other)
- 'LA22700-1' : 0 (Domestic partner)
- 'LA22697-9' : 1 (Legally separated)

**Marital status and living arrangements - Reported**
- skipping, most answers make no sense!

**Divorced**
- skipping, no answer

**Single person**
- skipping, question makes no sense!

### filter observation df to single living status

In [ ]:
single_obs = observation[observation['standard_concept_name'].str.contains('marital|divorced|single', case = False)]
single_obs['standard_concept_name'].unique()

In [ ]:
single_obs = single_obs[~single_obs['standard_concept_name'].str.contains('homeless|unknown|Deprecated|conflict|problems', case = False)]
single_obs['standard_concept_name'].unique()

### marital status

In [ ]:
single_obs_sub = single_obs[single_obs['standard_concept_name'].isin(['Marital status [NHANES]'])]
print(single_obs_sub['standard_concept_code'].unique())
print(single_obs_sub['standard_vocabulary'].unique())
single_obs_sub ['value_as_string'].value_counts()

In [ ]:
single_obs_sub.rename(columns = {'value_as_string' : 'SINGLE_STATUS'}, inplace = True)
single_obs_sub = single_obs_sub[['person_id', 'SINGLE_STATUS', 'observation_datetime']]
single_obs_sub['observation_datetime'] = pd.to_datetime(single_obs_sub['observation_datetime'])
single_obs_sub.sort_values(by = ['person_id', 'observation_datetime'], inplace = True, ascending = False)
print(len(single_obs_sub.index))
single_obs_sub.drop_duplicates(subset = 'person_id', keep = 'first', inplace = True)
print(len(single_obs_sub.index))
single_obs_sub.drop(columns = ['observation_datetime'], inplace = True)
single_obs_sub['SINGLE_STATUS'] = single_obs_sub['SINGLE_STATUS'].replace({'LA48-4' : 0,
                                                                           'LA17717-2' : 1,
                                                                           'LA47-6' : 1,
                                                                           'LA51-8' : 1,
                                                                           'LA49-2' : 1,
                                                                           'LA15605-1' : 0,
                                                                           'No matching concept': np.nan,
                                                                           'LA10058-8': np.nan,
                                                                           'LA4288-2' : 1,
                                                                           'LA4489-6': np.nan,
                                                                           'LA30381-0' : 0,
                                                                           'LA46-8' : np.nan,
                                                                           'LA22700-1' : 0,
                                                                           'LA22697-9' : 1})
single_obs_sub['SINGLE_STATUS'] = single_obs_sub['SINGLE_STATUS'].astype('Int64')
print(single_obs_sub['SINGLE_STATUS'].value_counts())
print(len(single_obs_sub.index))
print(len(single_obs_sub['person_id'].unique()))
single_obs_sub.head()

In [ ]:
single_obs_clean = single_obs_sub

### merge

In [ ]:
print(len(single_obs_clean['person_id'].unique()))
print(len(single_obs_clean.index))

In [ ]:
hf_single_obs = hf_education_obs.merge(single_obs_clean, on = 'person_id', how = 'left')
print(len(hf_single_obs.index))
print(hf_single_obs['SINGLE_STATUS'].value_counts(dropna = False))
hf_single_obs.head()

# clean survey data

## smoking

### encoding
- 0 = nonsmoker
- 1 = former smoker
- 2 = current smoker

**Smoking: Number Of Years**
- 'PMI: Dont Know' : np.nan (missing)
- 'PMI: Skip' : np.nan (missing)
- 'PMI: Prefer Not To Answer' : np.nan (missing)
- 0 = nonsmoker
- \> 0 = smoker

**Smoking: Smoke Frequency**
- 'Smoke Frequency: Not At All' : 0
- 'Smoke Frequency: Every Day' : 2
- 'Smoke Frequency: Some Days' : 2
- 'PMI: Skip' : np.nan (missing)
- 'PMI: Prefer Not To Answer' : np.nan (missing)
- 'PMI: Dont Know' : np.nan (missing)

**Smoking: 100 Cigs Lifetime**
- '100 Cigs Lifetime: No' : 0
- '100 Cigs Lifetime: Yes' : 2
- 'PMI: Skip' : np.nan (missing)
- 'PMI: Prefer Not To Answer' : np.nan (missing)
- 'PMI: Dont Know' : np.nan (missing)

**Cigar Smoking: Cigar Smoke Participant**
- 'Cigar Smoke Participant: No' : 0
- 'Cigar Smoke Participant: Yes' : 2
- 'PMI: Skip' : np.nan (missing)
- 'PMI: Prefer Not To Answer' : np.nan (missing)
- 'PMI: Dont Know' : np.nan (missing)

**Cigar Smoking: Current Cigar Frequency**
- 'Current Cigar Frequency: Not At All' : 0
- 'Current Cigar Frequency: Some Days' : 2
- 'Current Cigar Frequency: Every Day' : 2
- 'PMI: Skip' : np.nan (missing)
- 'PMI: Prefer Not To Answer' : np.nan (missing)
- 'PMI: Dont Know' : np.nan (missing)

**Smoking: Current Daily Cigarette Number**
- 'PMI: Dont Know' : np.nan (missing)
- 'PMI: Skip' : np.nan (missing)
- 'PMI: Prefer Not To Answer' : np.nan (missing)
- 0 = nonsmoker
- \> 0 = smoker

**Attempt Quit Smoking: Completely Quit Age**
- quit age > AGE : 2
- quit age < AGE : 1

### filter survey data to smoking

In [ ]:
smoking_survey = survey[survey['question'].str.contains('smoking', case = False)]
smoking_survey['question'].unique()

In [ ]:
smoking_survey = smoking_survey[~smoking_survey['question'].str.contains('serious|starting|hookah|electronic', case = False)]
smoking_survey['question'].unique()

### Smoking: Number Of Years

In [ ]:
smoking_survey_sub = smoking_survey[smoking_survey['question'].isin(['Smoking: Number Of Years'])]
print(smoking_survey_sub[smoking_survey_sub['answer'].str.contains('PMI')]['answer'].value_counts())
smoking_survey_sub[~smoking_survey_sub['answer'].str.contains('PMI')]['answer'].astype(int).describe()

In [ ]:
smoking_survey_sub.rename(columns = {'answer' : 'SMOKING_YEARS'}, inplace = True)
smoking_survey_sub = smoking_survey_sub[['person_id', 'SMOKING_YEARS']]
smoking_survey_sub['SMOKING_YEARS'] = smoking_survey_sub['SMOKING_YEARS'].replace({'PMI: Dont Know' : np.nan,
                                                                                   'PMI: Skip' : np.nan,
                                                                                   'PMI: Prefer Not To Answer' : np.nan})
smoking_survey_sub['SMOKING_YEARS'] = smoking_survey_sub['SMOKING_YEARS'].astype('Int64')
smoking_survey_sub['SMOKING_YEARS'] = smoking_survey_sub['SMOKING_YEARS'].apply(lambda x: 2 if pd.notna(x) and x > 0 else (0 if pd.notna(x) else pd.NA)).astype("Int64")
smoking_survey_sub['SMOKING_YEARS'] = smoking_survey_sub['SMOKING_YEARS'].astype('Int64')
print(smoking_survey_sub['SMOKING_YEARS'].value_counts(dropna = False))
print(len(smoking_survey_sub.index))
print(len(smoking_survey_sub['person_id'].unique()))
smoking_survey_sub.head()

In [ ]:
smoking_survey_clean = smoking_survey_sub

### smoke frequency

In [ ]:
smoking_survey_sub = smoking_survey[smoking_survey['question'].isin(['Smoking: Smoke Frequency'])]
print(smoking_survey_sub['answer'].value_counts())

In [ ]:
smoking_survey_sub.rename(columns = {'answer' : 'SMOKING_FREQ'}, inplace = True)
smoking_survey_sub = smoking_survey_sub[['person_id', 'SMOKING_FREQ']]
smoking_survey_sub['SMOKING_FREQ'] = smoking_survey_sub['SMOKING_FREQ'].replace({'Smoke Frequency: Not At All' : 0,
                                                                                   'Smoke Frequency: Every Day' : 2,
                                                                                   'Smoke Frequency: Some Days' : 2,
                                                                                   'PMI: Skip' : np.nan,
                                                                                   'PMI: Prefer Not To Answer' : np.nan,
                                                                                   'PMI: Dont Know' : np.nan})
smoking_survey_sub['SMOKING_FREQ'] = smoking_survey_sub['SMOKING_FREQ'].astype('Int64')
print(smoking_survey_sub['SMOKING_FREQ'].value_counts(dropna = False))
print(len(smoking_survey_sub.index))
print(len(smoking_survey_sub['person_id'].unique()))
smoking_survey_sub.head()

In [ ]:
smoking_survey_clean = smoking_survey_clean.merge(smoking_survey_sub, on = 'person_id', how = 'outer')
smoking_survey_clean.head()

### 100 cigs

In [ ]:
smoking_survey_sub = smoking_survey[smoking_survey['question'].isin(['Smoking: 100 Cigs Lifetime'])]
print(smoking_survey_sub['answer'].value_counts())

In [ ]:
smoking_survey_sub.rename(columns = {'answer' : 'SMOKING_100_CIGS'}, inplace = True)
smoking_survey_sub = smoking_survey_sub[['person_id', 'SMOKING_100_CIGS']]
smoking_survey_sub['SMOKING_100_CIGS'] = smoking_survey_sub['SMOKING_100_CIGS'].replace({'100 Cigs Lifetime: No' : 0,
                                                                                         '100 Cigs Lifetime: Yes' : 2,
                                                                                         'PMI: Skip' : np.nan,
                                                                                         'PMI: Prefer Not To Answer' : np.nan,
                                                                                         'PMI: Dont Know' : np.nan})
smoking_survey_sub['SMOKING_100_CIGS'] = smoking_survey_sub['SMOKING_100_CIGS'].astype('Int64')
print(smoking_survey_sub['SMOKING_100_CIGS'].value_counts(dropna = False))
print(len(smoking_survey_sub.index))
print(len(smoking_survey_sub['person_id'].unique()))
smoking_survey_sub.head()

In [ ]:
smoking_survey_clean = smoking_survey_clean.merge(smoking_survey_sub, on = 'person_id', how = 'outer')
smoking_survey_clean.head()

### cigar smoke participant

In [ ]:
smoking_survey_sub = smoking_survey[smoking_survey['question'].isin(['Cigar Smoking: Cigar Smoke Participant'])]
print(smoking_survey_sub['answer'].value_counts())

In [ ]:
smoking_survey_sub.rename(columns = {'answer' : 'SMOKING_CIGAR'}, inplace = True)
smoking_survey_sub = smoking_survey_sub[['person_id', 'SMOKING_CIGAR']]
smoking_survey_sub['SMOKING_CIGAR'] = smoking_survey_sub['SMOKING_CIGAR'].replace({'Cigar Smoke Participant: No' : 0,
                                                                                   'Cigar Smoke Participant: Yes' : 2,
                                                                                   'PMI: Skip' : np.nan,
                                                                                   'PMI: Prefer Not To Answer' : np.nan,
                                                                                   'PMI: Dont Know' : np.nan})
smoking_survey_sub['SMOKING_CIGAR'] = smoking_survey_sub['SMOKING_CIGAR'].astype('Int64')
print(smoking_survey_sub['SMOKING_CIGAR'].value_counts(dropna = False))
print(len(smoking_survey_sub.index))
print(len(smoking_survey_sub['person_id'].unique()))
smoking_survey_sub.head()

In [ ]:
smoking_survey_clean = smoking_survey_clean.merge(smoking_survey_sub, on = 'person_id', how = 'outer')
smoking_survey_clean.head()

### cigar freq

In [ ]:
smoking_survey_sub = smoking_survey[smoking_survey['question'].isin(['Cigar Smoking: Current Cigar Frequency'])]
print(smoking_survey_sub['answer'].value_counts())

In [ ]:
smoking_survey_sub.rename(columns = {'answer' : 'SMOKING_CIGAR_FREQ'}, inplace = True)
smoking_survey_sub = smoking_survey_sub[['person_id', 'SMOKING_CIGAR_FREQ']]
smoking_survey_sub['SMOKING_CIGAR_FREQ'] = smoking_survey_sub['SMOKING_CIGAR_FREQ'].replace({'Current Cigar Frequency: Not At All' : 0,
                                                                                   'Current Cigar Frequency: Some Days' : 2,
                                                                                   'Current Cigar Frequency: Every Day' : 2,
                                                                                   'PMI: Skip' : np.nan,
                                                                                   'PMI: Prefer Not To Answer' : np.nan,
                                                                                   'PMI: Dont Know' : np.nan})
smoking_survey_sub['SMOKING_CIGAR_FREQ'] = smoking_survey_sub['SMOKING_CIGAR_FREQ'].astype('Int64')
print(smoking_survey_sub['SMOKING_CIGAR_FREQ'].value_counts(dropna = False))
print(len(smoking_survey_sub.index))
print(len(smoking_survey_sub['person_id'].unique()))
smoking_survey_sub.head()

In [ ]:
smoking_survey_clean = smoking_survey_clean.merge(smoking_survey_sub, on = 'person_id', how = 'outer')
smoking_survey_clean.head()

### daily cig number

In [ ]:
smoking_survey_sub = smoking_survey[smoking_survey['question'].isin(['Smoking: Current Daily Cigarette Number'])]
print(smoking_survey_sub[smoking_survey_sub['answer'].str.contains('PMI')]['answer'].value_counts())
smoking_survey_sub[~smoking_survey_sub['answer'].str.contains('PMI')]['answer'].astype(int).describe()

In [ ]:
smoking_survey_sub.rename(columns = {'answer' : 'SMOKING_CIG_DAILY'}, inplace = True)
smoking_survey_sub = smoking_survey_sub[['person_id', 'SMOKING_CIG_DAILY']]
smoking_survey_sub['SMOKING_CIG_DAILY'] = smoking_survey_sub['SMOKING_CIG_DAILY'].replace({'PMI: Dont Know' : np.nan,
                                                                                   'PMI: Skip' : np.nan,
                                                                                   'PMI: Prefer Not To Answer' : np.nan})
smoking_survey_sub['SMOKING_CIG_DAILY'] = smoking_survey_sub['SMOKING_CIG_DAILY'].astype('Int64')
smoking_survey_sub['SMOKING_CIG_DAILY'] = smoking_survey_sub['SMOKING_CIG_DAILY'].apply(lambda x: 2 if pd.notna(x) and x > 0 else (0 if pd.notna(x) else pd.NA)).astype("Int64")
smoking_survey_sub['SMOKING_CIG_DAILY'] = smoking_survey_sub['SMOKING_CIG_DAILY'].astype('Int64')
print(smoking_survey_sub['SMOKING_CIG_DAILY'].value_counts(dropna = False))
print(len(smoking_survey_sub.index))
print(len(smoking_survey_sub['person_id'].unique()))
smoking_survey_sub.head()

In [ ]:
smoking_survey_clean = smoking_survey_clean.merge(smoking_survey_sub, on = 'person_id', how = 'outer')
smoking_survey_clean.head()

### age completely quit smoking

In [ ]:
smoking_survey_sub = smoking_survey[smoking_survey['question'].isin(['Attempt Quit Smoking: Completely Quit Age'])]
print(smoking_survey_sub[smoking_survey_sub['answer'].str.contains('PMI')]['answer'].value_counts())
smoking_survey_sub[~smoking_survey_sub['answer'].str.contains('PMI')]['answer'].astype(int).describe()

In [ ]:
print(len(smoking_survey_sub.index))
print(len(smoking_survey_sub['person_id'].unique()))

In [ ]:
smoking_survey_age = age_df.merge(smoking_survey_sub, on = 'person_id', how = 'inner')
print(len(smoking_survey_age.index))

In [ ]:
smoking_survey_age = smoking_survey_age[['person_id', 'answer', 'AGE']]
smoking_survey_age['answer'] = smoking_survey_age['answer'].replace({'PMI: Dont Know' : np.nan,
                                                                                   'PMI: Skip' : np.nan,
                                                                                   'PMI: Prefer Not To Answer' : np.nan})
smoking_survey_age['answer'] = smoking_survey_age['answer'].astype('Int64')
smoking_survey_age_no_missing = smoking_survey_age[smoking_survey_age['answer'].isna() == False]
smoking_survey_age_missing = smoking_survey_age[smoking_survey_age['answer'].isna() == True]
smoking_survey_age_missing['SMOKING_QUIT_AGE'] = np.nan
smoking_survey_age_no_missing['SMOKING_QUIT_AGE'] = np.where(smoking_survey_age_no_missing['answer'] < smoking_survey_age_no_missing['AGE'], 1, 2)
smoking_survey_age_no_missing['SMOKING_QUIT_AGE'] = smoking_survey_age_no_missing['SMOKING_QUIT_AGE'].astype('Int64')
smoking_survey_age = pd.concat([smoking_survey_age_no_missing, smoking_survey_age_missing])
smoking_survey_age.drop(columns = ['answer', 'AGE'], inplace = True)
print(smoking_survey_age['SMOKING_QUIT_AGE'].value_counts(dropna = False))
print(len(smoking_survey_age.index))
print(len(smoking_survey_age['person_id'].unique()))
smoking_survey_age.head()

In [ ]:
smoking_survey_clean = smoking_survey_clean.merge(smoking_survey_age, on = 'person_id', how = 'outer')
smoking_survey_clean.head()

### create combined column

In [ ]:
former_smoker = smoking_obs_clean[smoking_obs_clean['SMOKING_QUIT_AGE'] == 1]
print(len(former_smoker.index))
print(former_smoker['SMOKING_QUIT_AGE'].value_counts(dropna = False))
former_smoker['SMOKING_SURVEY'] = 1

In [ ]:
smoking_cols = [col for col in smoking_survey_clean.columns if col.startswith('SMOKING_')]
print(smoking_cols)
no_former_smoker = smoking_survey_clean[~smoking_survey_clean['person_id'].isin(former_smoker['person_id'])]
print(len(no_former_smoker.index))
no_former_smoker['SMOKING_SURVEY'] = no_former_smoker[smoking_cols].sum(axis=1)
print(no_former_smoker['SMOKING_SURVEY'].value_counts(dropna = False))
no_former_smoker.head()

In [ ]:
no_former_smoker['SMOKING_SURVEY'] = np.where(no_former_smoker['SMOKING_SURVEY']> 0, 2, 0)
no_former_smoker['SMOKING_SURVEY'] = no_former_smoker['SMOKING_SURVEY'].astype('Int64')
print(no_former_smoker['SMOKING_SURVEY'].value_counts(dropna = False))

In [ ]:
smoking_survey_clean_final = pd.concat([no_former_smoker, former_smoker], axis = 0)
print(len(smoking_survey_clean_final.index))
smoking_survey_clean_final['SMOKING_SURVEY'].value_counts(dropna = False)

In [ ]:
smoking_survey_clean_sub = smoking_survey_clean_final[['person_id', 'SMOKING_SURVEY']]
smoking_survey_clean_sub.head()

### merge

In [ ]:
smoking_obs_survey = smoking_obs_clean_sub.merge(smoking_survey_clean_sub, on = 'person_id', how = 'left')
print(len(smoking_obs_survey.index))
print(smoking_obs_survey[['SMOKING','SMOKING_SURVEY']].value_counts(dropna = False))
smoking_obs_survey.head()

In [ ]:
smoking_obs_survey.rename(columns = {'SMOKING' : 'SMOKING_OBS'}, inplace = True)

In [ ]:
former_smoker = smoking_obs_survey[((smoking_obs_survey['SMOKING_OBS'] == 1) | (smoking_obs_survey['SMOKING_OBS'].isna() == True)) & ((smoking_obs_survey['SMOKING_SURVEY'] == 1) | (smoking_obs_survey['SMOKING_SURVEY'].isna() == True) | (smoking_obs_survey['SMOKING_SURVEY'] == 0))]
print(len(former_smoker.index))
former_smoker['SMOKING'] = 1
former_smoker[['SMOKING_OBS','SMOKING_SURVEY']].value_counts(dropna = False)

In [ ]:
nonsmoker = smoking_obs_survey[((smoking_obs_survey['SMOKING_OBS'] == 0) | (smoking_obs_survey['SMOKING_OBS'].isna() == True)) & ((smoking_obs_survey['SMOKING_SURVEY'] == 0) | (smoking_obs_survey['SMOKING_SURVEY'].isna() == True))]
print(len(nonsmoker.index))
nonsmoker['SMOKING'] = 0
nonsmoker[['SMOKING_OBS','SMOKING_SURVEY']].value_counts(dropna = False)

In [ ]:
current_smoker = smoking_obs_survey[~smoking_obs_survey['person_id'].isin(former_smoker['person_id'])]
current_smoker = current_smoker[~current_smoker['person_id'].isin(nonsmoker['person_id'])]
current_smoker['SMOKING'] = 2
print(len(current_smoker.index))

In [ ]:
smoking_final = pd.concat([former_smoker, nonsmoker, current_smoker], axis = 0)
smoking_final = smoking_final[['person_id', 'SMOKING']]
print(len(smoking_final.index))
smoking_final['SMOKING'].value_counts(dropna = False)

### merge

In [ ]:
hf_smoking = hf_education_obs.drop(columns = ['SMOKING'])
hf_smoking = hf_smoking.merge(smoking_final, on = 'person_id', how = 'left')
print(len(hf_smoking.index))
print(hf_smoking['SMOKING'].isna().sum())

## income

### encoding
**Income: Annual Income**
- 'PMI: Prefer Not To Answer' : np.nan
- 'PMI: Skip' : np.nan
- 'Annual Income: less 10k' : 0
- 'Annual Income: 10k 25k' : 1
- 'Annual Income: 25k 35k' : 2
- 'Annual Income: 35k 50k' : 3
- 'Annual Income: 50k 75k' : 4
- 'Annual Income: 75k 100k' : 5
- 'Annual Income: 100k 150k' : 6
- 'Annual Income: 150k 200k' : 7
- 'Annual Income: more 200k' : 8

### filter survey data to income

In [ ]:
income_survey = survey[survey['question'].str.contains('income', case = False)]
income_survey['question'].unique()

### annual income

In [ ]:
income_survey_sub = income_survey[income_survey['question'].isin(['Income: Annual Income'])]
income_survey_sub['answer'].value_counts()

In [ ]:
income_survey_sub.rename(columns = {'answer' : 'INCOME_ANNUAL_SURVEY'}, inplace = True)
income_survey_sub = income_survey_sub[['person_id', 'INCOME_ANNUAL_SURVEY']]
income_survey_sub['INCOME_ANNUAL_SURVEY'] = income_survey_sub['INCOME_ANNUAL_SURVEY'].replace({'PMI: Prefer Not To Answer' : np.nan,
                                                                                               'PMI: Skip' : np.nan,
                                                                                               'Annual Income: less 10k' : 0,
                                                                                               'Annual Income: 10k 25k' : 1,
                                                                                               'Annual Income: 25k 35k' : 2,
                                                                                               'Annual Income: 35k 50k' : 3,
                                                                                               'Annual Income: 50k 75k' : 4,
                                                                                               'Annual Income: 75k 100k' : 5,
                                                                                               'Annual Income: 100k 150k' : 6,
                                                                                               'Annual Income: 150k 200k' : 7,
                                                                                               'Annual Income: more 200k' : 8})
income_survey_sub['INCOME_ANNUAL_SURVEY'] = income_survey_sub['INCOME_ANNUAL_SURVEY'].astype('Int64')
income_survey_sub['INCOME_ANNUAL_SURVEY'] = income_survey_sub['INCOME_ANNUAL_SURVEY'].astype('Int64')
print(income_survey_sub['INCOME_ANNUAL_SURVEY'].value_counts(dropna = False))
print(len(income_survey_sub.index))
print(len(income_survey_sub['person_id'].unique()))
income_survey_sub.head()

In [ ]:
income_survey_clean = income_survey_sub

### merge

In [ ]:
hf_income_survey = hf_smoking.merge(income_survey_clean, on = 'person_id', how = 'left')
print(len(hf_income_survey.index))
print(hf_income_survey['INCOME_ANNUAL_SURVEY'].value_counts(dropna = False))
hf_income_survey.head()

## education

### encoding
**Education Level: Highest Grade**
- 'Highest Grade: Never Attended' : 0
- 'Highest Grade: One Through Four' : 1
- 'Highest Grade: Five Through Eight' : 2
- 'Highest Grade: Nine Through Eleven' : 3
- 'Highest Grade: Twelve Or GED' : 4
- 'Highest Grade: College One to Three' : 5
- 'Highest Grade: College Graduate' : 6
- 'Highest Grade: Advanced Degree' : 7
- 'PMI: Skip' : np.nan
- 'PMI: Prefer Not To Answer' : np.nan

### filter survey data to education

In [ ]:
education_survey = survey[survey['question'].str.contains('education', case = False)]
education_survey['question'].unique()

### highest level education

In [ ]:
education_survey_sub = education_survey[education_survey['question'].isin(['Education Level: Highest Grade'])]
education_survey_sub['answer'].value_counts()

In [ ]:
education_survey_sub.rename(columns = {'answer' : 'EDUCATION_HIGHEST_SURVEY'}, inplace = True)
education_survey_sub = education_survey_sub[['person_id', 'EDUCATION_HIGHEST_SURVEY']]
education_survey_sub['EDUCATION_HIGHEST_SURVEY'] = education_survey_sub['EDUCATION_HIGHEST_SURVEY'].replace({'Highest Grade: Never Attended' : 0,
                                                                                                             'Highest Grade: One Through Four' : 1,
                                                                                                             'Highest Grade: Five Through Eight' : 2,
                                                                                                             'Highest Grade: Nine Through Eleven' : 3,
                                                                                                             'Highest Grade: Twelve Or GED' : 4,
                                                                                                             'Highest Grade: College One to Three' : 5,
                                                                                                             'Highest Grade: College Graduate' : 6,
                                                                                                             'Highest Grade: Advanced Degree' : 7,
                                                                                                             'PMI: Skip' : np.nan,
                                                                                                             'PMI: Prefer Not To Answer' : np.nan})
education_survey_sub['EDUCATION_HIGHEST_SURVEY'] = education_survey_sub['EDUCATION_HIGHEST_SURVEY'].astype('Int64')
print(education_survey_sub['EDUCATION_HIGHEST_SURVEY'].value_counts(dropna = False))
print(len(education_survey_sub.index))
print(len(education_survey_sub['person_id'].unique()))
education_survey_sub.head()

In [ ]:
education_survey_clean = education_survey_sub

### merge

In [ ]:
hf_education_survey = hf_income_survey.merge(education_survey_clean, on = 'person_id', how = 'left')
print(len(hf_education_survey.index))
print(hf_education_survey['EDUCATION_HIGHEST_SURVEY'].value_counts(dropna = False))
hf_education_survey.head()

## neighborhood

### encoding
**What is the main type of housing in your neighborhood?**
- 'Detached single-family housing' : 4
- 'Mix of single-family residences and townhouses' : 2
- 'Townhouses' : 3
- 'Apartments or condos of 4-12 stories' : 1
- 'Apartments or condos of more than 12 stories' : 0
- 'PMI: Dont Know' : np.nan
- 'PMI: Skip' : np.nan

**How much you agree or disagree that your neighborhood is safe?**
- 'Agree' : 2,
- 'Strongly agree' : 3,
- 'Disagree' : 1,
- 'PMI: Skip' : np.nan,
- 'Strongly disagree' : 0

**How much you agree or disagree that your neighborhood is clean?**
- 'Agree' : 2
- 'Strongly agree' : 3
- 'Disagree' : 1
- 'PMI: Skip': np.nan,
- 'Strongly disagree' : 0

**How much you agree or disagree that your neighborhood is noisy?**
- 'Agree' : 1,
- 'Strongly agree' : 0,
- 'Disagree' : 2,
- 'PMI: Skip': np.nan,
- 'Strongly disagree' : 3

**How much you agree or disagree that vandalism is common in your neighborhood?**
- 'Agree' : 1,
- 'Strongly agree' : 0,
- 'Disagree' : 2,
- 'PMI: Skip': np.nan,
- 'Strongly disagree' : 3

**How much you agree or disagree that people in your neighborhood can be trusted?**
- 'Agree' : 3,
- 'Neutral (neither agree nor disagree)' : 2,
- 'Strongly agree' : 4,
- 'Disagree' : 1
- 'PMI: Skip' : np.nan,
- 'Strongly disagree' : 0

**How much you agree or disagree that there is a lot of crime in your neighborhood?**
- 'Agree' : 1,
- 'Strongly agree' : 0,
- 'Disagree' : 2,
- 'PMI: Skip': np.nan,
- 'Strongly disagree' : 3

**How much you agree or disagree that there is a lot of graffiti in your neighborhood?**
- 'Agree' : 1,
- 'Strongly agree' : 0,
- 'Disagree' : 2,
- 'PMI: Skip': np.nan,
- 'Strongly disagree' : 3

**How much you agree or disagree that there is too much drug use in your neighborhood?**
- 'Agree' : 1,
- 'Strongly agree' : 0,
- 'Disagree' : 2,
- 'PMI: Skip': np.nan,
- 'Strongly disagree' : 3

**How much you agree or disagree that people in your neighborhood share the same values?**
- 'Agree' : 3,
- 'Neutral (neither agree nor disagree)' : 2,
- 'Strongly agree' : 4,
- 'Disagree' : 1,
- 'PMI: Skip' : np.nan,
- 'Strongly disagree' : 0

**How much you agree or disagree that there is too much alcohol use in your neighborhood?**
- 'Agree' : 1,
- 'Strongly agree' : 0,
- 'Disagree' : 2,
- 'PMI: Skip': np.nan,
- 'Strongly disagree' : 3

**There are sidewalks on most of the streets in my neighborhood. Would you say that you...**
- 'Strongly agree' : 3,
- 'Strongly disagree' : 0,
- 'Somewhat agree' : 2,
- 'Somewhat disagree' : 1,
- 'Does not apply to my neighborhood' : np.nan
- 'PMI: Skip': np.nan,
- 'PMI: Dont Know': np.nan

**How much you agree or disagree that in your neighborhood people watch out for each other?**
- 'Agree' : 3,
- 'Neutral (neither agree nor disagree)' : 2,
- 'Strongly agree' : 4,
- 'Disagree' : 1,
- 'PMI: Skip' : np.nan,
- 'Strongly disagree' : 0

**How much you agree or disagree that there are lot of abandoned buildings in your neighborhood?**
- 'Agree' : 1,
- 'Strongly agree' : 0,
- 'Disagree' : 2,
- 'PMI: Skip': np.nan,
- 'Strongly disagree' : 3

**How much you agree or disagree that people in your neighborhood generally get along with each other?**
- 'Agree' : 3,
- 'Neutral (neither agree nor disagree)' : 2,
- 'Strongly agree' : 4,
- 'Disagree' : 1,
- 'PMI: Skip' : np.nan,
- 'Strongly disagree' : 0

**The crime rate in my neighborhood makes it unsafe to go on walks at night. Would you say that you...**
- 'Strongly agree' : 0,
- 'Strongly disagree' : 3,
- 'Somewhat agree' : 1,
- 'Somewhat disagree' : 2,
- 'PMI: Skip': np.nan,
- 'PMI: Dont Know': np.nan

**The crime rate in my neighborhood makes it unsafe to go on walks during the day. Would you say that you...**
- 'Strongly agree' : 0,
- 'Strongly disagree' : 3,
- 'Somewhat agree' : 1,
- 'Somewhat disagree' : 2,
- 'PMI: Skip': np.nan,
- 'PMI: Dont Know': np.nan

**How much you agree or disagree that people in your neighborhood take good care of their houses and apartments?**
- 'Agree' : 2,
- 'Strongly agree' : 3,
- 'Disagree' : 1,
- 'PMI: Skip': np.nan,
- 'Strongly disagree' : 0

**There are facilities to bicycle in or near my neighborhood, such as special lanes, separate paths or trails, or shared use paths for cycles and pedestrians. Would you say that you...**
- 'Strongly agree' : 3,
- 'Strongly disagree' : 0,
- 'Somewhat agree' : 2,
- 'Somewhat disagree' : 1,
- 'Does not apply to my neighborhood' : np.nan,
- 'PMI: Skip': np.nan,
- 'PMI: Dont Know': np.nan

**My neighborhood has several free or low-cost recreation facilities, such as parks, walking trails, bike paths, recreation centers, playgrounds, public swimming pools, etc. Would you say that you...**
- 'Strongly agree' : 3,
- 'Strongly disagree' : 0,
- 'Somewhat agree' : 2,
- 'Somewhat disagree' : 1,
- 'PMI: Skip': np.nan,
- 'PMI: Dont Know': np.nan

**How much you agree or disagree that there are too many people hanging around on the streets near your home?**
- 'Strongly disagree' : 3,
- 'Disagree' : 2,
- 'Agree' : 1,
- 'PMI: Skip' : np.nan,
- 'Strongly agree' : 0

**How much you agree or disagree that you are always having trouble with your neighbors?**
- 'Strongly disagree' : 3,
- 'Disagree' : 2,
- 'PMI: Skip' : np.nan,
- 'Agree' : 1,
- 'Strongly agree' : 0

**Many shops, stores, markets or other places to buy things I need are within easy walking distance of my home. Would you say that you...**
- 'Strongly disagree' : 0,
- 'Strongly agree' : 3,
- 'Somewhat agree' : 2,
- 'Somewhat disagree' : 1,
- 'PMI: Skip' : np.nan,
- 'PMI: Dont Know': np.nan

**It is within a 10-15 minute walk to a transit stop (such as bus, train, trolley, or tram) from my home. Would you say that you...**
- 'Strongly disagree' : 0,
- 'Strongly agree' : 3,
- 'Somewhat agree' : 2,
- 'Somewhat disagree' : 1,
- 'PMI: Skip' : np.nan
- 'PMI: Dont Know': np.nan

### filter survey data to neighborhood

In [ ]:
neighbor_survey = survey[survey['question'].str.contains('neighbor|transit|store|street', case = False)]
neighbor_survey['question'].unique()

### type of housing

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['What is the main type of housing in your neighborhood?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_HOUSING_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_HOUSING_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_HOUSING_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_HOUSING_SURVEY'].replace({'Detached single-family housing' : 4,
                                                                                                                 'Mix of single-family residences and townhouses' : 2,
                                                                                                                 'Townhouses' : 3,
                                                                                                                 'Apartments or condos of 4-12 stories' : 1,
                                                                                                                 'Apartments or condos of more than 12 stories' : 0,
                                                                                                                 'PMI: Dont Know' : np.nan,
                                                                                                                 'PMI: Skip' : np.nan})
neighbor_survey_sub['NEIGHBORHOOD_HOUSING_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_HOUSING_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_HOUSING_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_sub

### safe

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that your neighborhood is safe?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_SAFE_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_SAFE_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_SAFE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_SAFE_SURVEY'].replace({'Agree' : 2,
                                                                                                           'Strongly agree' : 3,
                                                                                                           'Disagree' : 1,
                                                                                                           'PMI: Skip' : np.nan,
                                                                                                           'Strongly disagree' : 0})
neighbor_survey_sub['NEIGHBORHOOD_SAFE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_SAFE_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_SAFE_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### clean

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that your neighborhood is clean?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_CLEAN_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_CLEAN_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_CLEAN_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_CLEAN_SURVEY'].replace({'Agree' : 2,
                                                                                                           'Strongly agree' : 3,
                                                                                                           'Disagree' : 1,
                                                                                                           'PMI: Skip': np.nan,
                                                                                                           'Strongly disagree' : 0})
neighbor_survey_sub['NEIGHBORHOOD_CLEAN_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_CLEAN_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_CLEAN_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### noisy

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that your neighborhood is noisy?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_NOISE_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_NOISE_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_NOISE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_NOISE_SURVEY'].replace({'Agree' : 1,
                                                                                                             'Strongly agree' : 0,
                                                                                                             'Disagree' : 2,
                                                                                                             'PMI: Skip': np.nan,
                                                                                                             'Strongly disagree' : 3})
neighbor_survey_sub['NEIGHBORHOOD_NOISE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_NOISE_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_NOISE_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### vandalism

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that vandalism is common in your neighborhood?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_VANDALISM_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_VANDALISM_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_VANDALISM_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_VANDALISM_SURVEY'].replace({'Agree' : 1,
                                                                                                             'Strongly agree' : 0,
                                                                                                             'Disagree' : 2,
                                                                                                             'PMI: Skip': np.nan,
                                                                                                             'Strongly disagree' : 3})
neighbor_survey_sub['NEIGHBORHOOD_VANDALISM_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_VANDALISM_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_VANDALISM_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### people can be trusted

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that people in your neighborhood can be trusted?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_TRUST_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_TRUST_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_TRUST_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_TRUST_SURVEY'].replace({'Agree' : 3,
                                                                                                             'Neutral (neither agree nor disagree)' : 2,
                                                                                                             'Strongly agree' : 4,
                                                                                                             'Disagree' : 1,
                                                                                                             'PMI: Skip' : np.nan,
                                                                                                             'Strongly disagree' : 0})
neighbor_survey_sub['NEIGHBORHOOD_TRUST_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_TRUST_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_TRUST_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### a lot of crime

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that there is a lot of crime in your neighborhood?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_ALOT_CRIME_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_ALOT_CRIME_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_ALOT_CRIME_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_ALOT_CRIME_SURVEY'].replace({'Agree' : 1,
                                                                                                                       'Strongly agree' : 0,
                                                                                                                       'Disagree' : 2,
                                                                                                                       'PMI: Skip': np.nan,
                                                                                                                       'Strongly disagree' : 3})
neighbor_survey_sub['NEIGHBORHOOD_ALOT_CRIME_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_ALOT_CRIME_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_ALOT_CRIME_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### graffiti

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that there is a lot of graffiti in your neighborhood?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_GRAFFITI_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_GRAFFITI_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_GRAFFITI_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_GRAFFITI_SURVEY'].replace({'Agree' : 1,
                                                                                                                       'Strongly agree' : 0,
                                                                                                                       'Disagree' : 2,
                                                                                                                       'PMI: Skip': np.nan,
                                                                                                                       'Strongly disagree' : 3})
neighbor_survey_sub['NEIGHBORHOOD_GRAFFITI_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_GRAFFITI_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_GRAFFITI_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### drug use

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that there is too much drug use in your neighborhood?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_DRUG_USE_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_DRUG_USE_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_DRUG_USE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_DRUG_USE_SURVEY'].replace({'Agree' : 1,
                                                                                                                       'Strongly agree' : 0,
                                                                                                                       'Disagree' : 2,
                                                                                                                       'PMI: Skip': np.nan,
                                                                                                                       'Strongly disagree' : 3})
neighbor_survey_sub['NEIGHBORHOOD_DRUG_USE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_DRUG_USE_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_DRUG_USE_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### share same values

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that people in your neighborhood share the same values?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_SAME_VALUES_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_SAME_VALUES_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_SAME_VALUES_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_SAME_VALUES_SURVEY'].replace({'Agree' : 3,
                                                                                                                         'Neutral (neither agree nor disagree)' : 2,
                                                                                                                         'Strongly agree' : 4,
                                                                                                                         'Disagree' : 1,
                                                                                                                         'PMI: Skip' : np.nan,
                                                                                                                         'Strongly disagree' : 0})
neighbor_survey_sub['NEIGHBORHOOD_SAME_VALUES_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_SAME_VALUES_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_SAME_VALUES_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### alcohol use

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that there is too much alcohol use in your neighborhood?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_ALCOHOL_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_ALCOHOL_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_ALCOHOL_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_ALCOHOL_SURVEY'].replace({'Agree' : 1,
                                                                                                                         'Strongly agree' : 0,
                                                                                                                         'Disagree' : 2,
                                                                                                                         'PMI: Skip': np.nan,
                                                                                                                         'Strongly disagree' : 3})
neighbor_survey_sub['NEIGHBORHOOD_ALCOHOL_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_ALCOHOL_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_ALCOHOL_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### sidewalks

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['There are sidewalks on most of the streets in my neighborhood. Would you say that you...'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_SIDEWALK_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_SIDEWALK_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_SIDEWALK_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_SIDEWALK_SURVEY'].replace({'Strongly agree' : 3,
                                                                                                                   'Strongly disagree' : 0,
                                                                                                                   'Somewhat agree' : 2,
                                                                                                                   'Somewhat disagree' : 1,
                                                                                                                   'Does not apply to my neighborhood' : np.nan,
                                                                                                                   'PMI: Skip': np.nan,
                                                                                                                   'PMI: Dont Know': np.nan})
neighbor_survey_sub['NEIGHBORHOOD_SIDEWALK_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_SIDEWALK_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_SIDEWALK_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### people watch out for each other

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that in your neighborhood people watch out for each other?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_WATCH_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_WATCH_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_WATCH_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_WATCH_SURVEY'].replace({'Agree' : 3,
                                                                                                             'Neutral (neither agree nor disagree)' : 2,
                                                                                                             'Strongly agree' : 4,
                                                                                                             'Disagree' : 1,
                                                                                                             'PMI: Skip' : np.nan,
                                                                                                             'Strongly disagree' : 0})
neighbor_survey_sub['NEIGHBORHOOD_WATCH_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_WATCH_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_WATCH_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### abandoned buildings

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that there are lot of abandoned buildings in your neighborhood?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_BUILDING_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_BUILDING_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_BUILDING_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_BUILDING_SURVEY'].replace({'Agree' : 1,
                                                                                                                   'Strongly agree' : 0,
                                                                                                                   'Disagree' : 2,
                                                                                                                   'PMI: Skip': np.nan,
                                                                                                                   'Strongly disagree' : 3})
neighbor_survey_sub['NEIGHBORHOOD_BUILDING_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_BUILDING_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_BUILDING_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### get along

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that people in your neighborhood generally get along with each other?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_GET_ALONG_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_GET_ALONG_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_GET_ALONG_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_GET_ALONG_SURVEY'].replace({'Agree' : 3,
                                                                                                                     'Neutral (neither agree nor disagree)' : 2,
                                                                                                                     'Strongly agree' : 4,
                                                                                                                     'Disagree' : 1,
                                                                                                                     'PMI: Skip' : np.nan,
                                                                                                                     'Strongly disagree' : 0})
neighbor_survey_sub['NEIGHBORHOOD_GET_ALONG_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_GET_ALONG_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_GET_ALONG_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### unsafe to walk at night

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['The crime rate in my neighborhood makes it unsafe to go on walks at night. Would you say that you...'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_UNSAFE_WALK_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_UNSAFE_WALK_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_UNSAFE_WALK_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_UNSAFE_WALK_SURVEY'].replace({'Strongly agree' : 0,
                                                                                                                         'Strongly disagree' : 3,
                                                                                                                         'Somewhat agree' : 1,
                                                                                                                         'Somewhat disagree' : 2,
                                                                                                                         'PMI: Skip': np.nan,
                                                                                                                         'PMI: Dont Know': np.nan})
neighbor_survey_sub['NEIGHBORHOOD_UNSAFE_WALK_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_UNSAFE_WALK_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_UNSAFE_WALK_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### unsafe to walk during the day due to crime

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['The crime rate in my neighborhood makes it unsafe to go on walks during the day. Would you say that you...'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_CRIME_WALK_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_CRIME_WALK_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_CRIME_WALK_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_CRIME_WALK_SURVEY'].replace({'Strongly agree' : 0,
                                                                                                                         'Strongly disagree' : 3,
                                                                                                                         'Somewhat agree' : 1,
                                                                                                                         'Somewhat disagree' : 2,
                                                                                                                         'PMI: Skip': np.nan,
                                                                                                                         'PMI: Dont Know': np.nan})
neighbor_survey_sub['NEIGHBORHOOD_CRIME_WALK_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_CRIME_WALK_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_CRIME_WALK_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### take care of homes

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that people in your neighborhood take good care of their houses and apartments?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_CARE_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_CARE_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_CARE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_CARE_SURVEY'].replace({'Agree' : 2,
                                                                                                           'Strongly agree' : 3,
                                                                                                           'Disagree' : 1,
                                                                                                           'PMI: Skip': np.nan,
                                                                                                           'Strongly disagree' : 0})
neighbor_survey_sub['NEIGHBORHOOD_CARE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_CARE_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_CARE_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### bike facilities

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['There are facilities to bicycle in or near my neighborhood, such as special lanes, separate paths or trails, or shared use paths for cycles and pedestrians. Would you say that you...'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_BIKE_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_BIKE_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_BIKE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_BIKE_SURVEY'].replace({'Strongly agree' : 3,
                                                                                                           'Strongly disagree' : 0,
                                                                                                           'Somewhat agree' : 2,
                                                                                                           'Somewhat disagree' : 1,
                                                                                                           'Does not apply to my neighborhood' : np.nan,
                                                                                                           'PMI: Skip': np.nan,
                                                                                                           'PMI: Dont Know': np.nan})
neighbor_survey_sub['NEIGHBORHOOD_BIKE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_BIKE_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_BIKE_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### free recreational amenities

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['My neighborhood has several free or low-cost recreation facilities, such as parks, walking trails, bike paths, recreation centers, playgrounds, public swimming pools, etc. Would you say that you...'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_FREE_AMENITIES_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_FREE_AMENITIES_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_FREE_AMENITIES_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_FREE_AMENITIES_SURVEY'].replace({'Strongly agree' : 3,
                                                                                                           'Strongly disagree' : 0,
                                                                                                           'Somewhat agree' : 2,
                                                                                                           'Somewhat disagree' : 1,
                                                                                                           'PMI: Skip': np.nan,
                                                                                                           'PMI: Dont Know': np.nan})
neighbor_survey_sub['NEIGHBORHOOD_FREE_AMENITIES_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_FREE_AMENITIES_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_FREE_AMENITIES_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### people hanging around streets

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that there are too many people hanging around on the streets near your home?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_PPL_HANGING_AROUND_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_PPL_HANGING_AROUND_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_PPL_HANGING_AROUND_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_PPL_HANGING_AROUND_SURVEY'].replace({'Strongly disagree' : 3,
                                                                                                                                       'Disagree' : 2,
                                                                                                                                       'Agree' : 1,
                                                                                                                                       'PMI: Skip' : np.nan,
                                                                                                                                       'Strongly agree' : 0})
neighbor_survey_sub['NEIGHBORHOOD_PPL_HANGING_AROUND_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_PPL_HANGING_AROUND_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_PPL_HANGING_AROUND_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### trouble

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['How much you agree or disagree that you are always having trouble with your neighbors?'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_TROUBLE_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_TROUBLE_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_TROUBLE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_TROUBLE_SURVEY'].replace({'Strongly disagree' : 3,
                                                                                                                 'Disagree' : 2,
                                                                                                                 'PMI: Skip' : np.nan,
                                                                                                                 'Agree' : 1,
                                                                                                                 'Strongly agree' : 0})
neighbor_survey_sub['NEIGHBORHOOD_TROUBLE_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_TROUBLE_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_TROUBLE_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### stores

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['Many shops, stores, markets or other places to buy things I need are within easy walking distance of my home. Would you say that you...'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_STORES_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_STORES_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_STORES_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_STORES_SURVEY'].replace({'Strongly disagree' : 0,
                                                                                                               'Strongly agree' : 3,
                                                                                                               'Somewhat agree' : 2,
                                                                                                               'Somewhat disagree' : 1,
                                                                                                               'PMI: Skip' : np.nan,
                                                                                                               'PMI: Dont Know': np.nan})
neighbor_survey_sub['NEIGHBORHOOD_STORES_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_STORES_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_STORES_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### transit

In [ ]:
neighbor_survey_sub = neighbor_survey[neighbor_survey['question'].isin(['It is within a 10-15 minute walk to a transit stop (such as bus, train, trolley, or tram) from my home. Would you say that you...'])]
neighbor_survey_sub['answer'].value_counts()

In [ ]:
neighbor_survey_sub.rename(columns = {'answer' : 'NEIGHBORHOOD_TRANSIT_SURVEY'}, inplace = True)
neighbor_survey_sub = neighbor_survey_sub[['person_id', 'NEIGHBORHOOD_TRANSIT_SURVEY']]
neighbor_survey_sub['NEIGHBORHOOD_TRANSIT_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_TRANSIT_SURVEY'].replace({'Strongly disagree' : 0,
                                                                                                                 'Strongly agree' : 3,
                                                                                                                 'Somewhat agree' : 2,
                                                                                                                 'Somewhat disagree' : 1,
                                                                                                                 'PMI: Skip' : np.nan,
                                                                                                                 'PMI: Dont Know': np.nan})
neighbor_survey_sub['NEIGHBORHOOD_TRANSIT_SURVEY'] = neighbor_survey_sub['NEIGHBORHOOD_TRANSIT_SURVEY'].astype('Int64')
print(neighbor_survey_sub['NEIGHBORHOOD_TRANSIT_SURVEY'].value_counts(dropna = False))
print(len(neighbor_survey_sub.index))
print(len(neighbor_survey_sub['person_id'].unique()))
neighbor_survey_sub.head()

In [ ]:
neighbor_survey_clean = neighbor_survey_clean.merge(neighbor_survey_sub, on = 'person_id', how = 'outer')
neighbor_survey_clean.head()

### merge

In [ ]:
hf_neighbor_survey = hf_education_survey.merge(neighbor_survey_clean, on = 'person_id', how = 'left')
print(len(hf_neighbor_survey.index))
hf_neighbor_survey.head()

# clean AOU SDOH file

## check for duplicates (no duplicates)

In [ ]:
print(len(zip_code_socioeconomic.index))
print(len(zip_code_socioeconomic['person_id'].unique()))

## drop extra columns

In [ ]:
zip_code_socioeconomic_sub = zip_code_socioeconomic[['person_id',
                                                     'median_income',
                                                     'deprivation_index']]
zip_code_socioeconomic_sub.head()

## rename columns

In [ ]:
zip_code_socioeconomic_sub.rename(columns = {'median_income': 'CENSUS_MEDIAN_INCOME',
                                             'deprivation_index' : 'SOCIAL_DEPRIVATION_INDEX'},
                                  inplace = True)
zip_code_socioeconomic_sub.head()

## merge

In [ ]:
hf_ses = hf_neighbor_survey.merge(zip_code_socioeconomic_sub, how = 'left', on = 'person_id')
print(len(hf_ses.index))
print(len(hf_ses['CENSUS_MEDIAN_INCOME'].dropna().index))
print(len(hf_ses['SOCIAL_DEPRIVATION_INDEX'].dropna().index))
hf_ses.head()

# check if columns derived from observation and survey data are equal

## neighborhood columns

### neighborhood columns that don't match
**NEIGHBORHOOD_WATCH != NEIGHBORHOOD_WATCH_SURVEY**
- fixed. there are no neutrals in this df. so use the observation column. when you convert the survey column to the same encoding as observation, they are equal

*NEIGHBORHOOD_WATCH (oberservation encoding): How much you agree or disagree that in your neighborhood people watch out for each other?*
- 'LA15774-5' : 2 (Agree)
- 'LA15237-3' : 3 (Strongly agree)
- 'LA15773-7' : 1 (Disagree)
- 'LA15236-5' : 0 (Strongly disagree)
- 'PMI_Skip' : np.nan (Missing)

*NEIGHBORHOOD_WATCH (survey encoding): How much you agree or disagree that in your neighborhood people watch out for each other?*
- 'Agree' : 3,
- 'Neutral (neither agree nor disagree)' : 2,
- 'Strongly agree' : 4,
- 'Disagree' : 1,
- 'PMI: Skip' : np.nan,
- 'Strongly disagree' : 0

**NEIGHBORHOOD_GET_ALONG != NEIGHBORHOOD_GET_ALONG_SURVEY**
- opposite questions. the opposite answers check out- use either question

*NEIGHBORHOOD_GET_ALONG (observation encoding): People in this neighborhood generally don't get along with each other [PhenX]*
- 'LA15774-5' : 1 (Agree)
- 'LA15237-3' : 0 (Strongly agree)
- 'LA15240-7' : 2 (Neither agree nor disagree)
- 'LA15773-7' : 3 (Disagree)
- 'PMI_Skip' : np.nan (Missing)
- 'LA15236-5' : 4 (Strongly disagree)

*NEIGHBORHOOD_GET_ALONG (survey encoding):How much you agree or disagree that people in your neighborhood generally get along with each other?*
- 'Agree' : 3,
- 'Neutral (neither agree nor disagree)' : 2,
- 'Strongly agree' : 4,
- 'Disagree' : 1,
- 'PMI: Skip' : np.nan,
- 'Strongly disagree' : 0

### columns that match

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_SAFE_CRIME'] != hf_neighbor_survey['NEIGHBORHOOD_SAFE_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_FREE_AMENITIES'] != hf_neighbor_survey['NEIGHBORHOOD_FREE_AMENITIES_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_DRUG_USE'] != hf_neighbor_survey['NEIGHBORHOOD_DRUG_USE_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_TRUST'] != hf_neighbor_survey['NEIGHBORHOOD_TRUST_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_BUILDING'] != hf_neighbor_survey['NEIGHBORHOOD_BUILDING_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_ALCOHOL'] != hf_neighbor_survey['NEIGHBORHOOD_ALCOHOL_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_VANDALISM'] != hf_neighbor_survey['NEIGHBORHOOD_VANDALISM_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_SIDEWALK'] != hf_neighbor_survey['NEIGHBORHOOD_SIDEWALK_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_BIKE'] != hf_neighbor_survey['NEIGHBORHOOD_BIKE_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_CARE'] != hf_neighbor_survey['NEIGHBORHOOD_CARE_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_CLEAN'] != hf_neighbor_survey['NEIGHBORHOOD_CLEAN_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_HOUSING'] != hf_neighbor_survey['NEIGHBORHOOD_HOUSING_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_GRAFFITI'] != hf_neighbor_survey['NEIGHBORHOOD_GRAFFITI_SURVEY']][['person_id','NEIGHBORHOOD_GRAFFITI','NEIGHBORHOOD_GRAFFITI_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_NOISE'] != hf_neighbor_survey['NEIGHBORHOOD_NOISE_SURVEY']][['person_id','NEIGHBORHOOD_NOISE','NEIGHBORHOOD_NOISE_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_UNSAFE_WALK'] != hf_neighbor_survey['NEIGHBORHOOD_UNSAFE_WALK_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_CRIME_WALK'] != hf_neighbor_survey['NEIGHBORHOOD_CRIME_WALK_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_ALOT_CRIME'] != hf_neighbor_survey['NEIGHBORHOOD_ALOT_CRIME_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_SAME_VALUES'] != hf_neighbor_survey['NEIGHBORHOOD_SAME_VALUES_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_PPL_HANGING_AROUND'] != hf_neighbor_survey['NEIGHBORHOOD_PPL_HANGING_AROUND_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_TROUBLE'] != hf_neighbor_survey['NEIGHBORHOOD_TROUBLE_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_STORES'] != hf_neighbor_survey['NEIGHBORHOOD_STORES_SURVEY']]

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_TRANSIT'] != hf_neighbor_survey['NEIGHBORHOOD_TRANSIT_SURVEY']]

### columns that don't match

#### watch out for each other

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_WATCH'] != hf_neighbor_survey['NEIGHBORHOOD_WATCH_SURVEY']][['person_id','NEIGHBORHOOD_WATCH','NEIGHBORHOOD_WATCH_SURVEY']]

In [ ]:
hf_neighbor_survey['NEIGHBORHOOD_WATCH_SURVEY'].value_counts()

In [ ]:
hf_neighbor_survey['NEIGHBORHOOD_WATCH_SURVEY_dup'] = hf_neighbor_survey['NEIGHBORHOOD_WATCH_SURVEY']
hf_neighbor_survey['NEIGHBORHOOD_WATCH_SURVEY_dup'] = hf_neighbor_survey['NEIGHBORHOOD_WATCH_SURVEY_dup'].replace({3 : 2,
                                                                                                          4 : 3})
hf_neighbor_survey['NEIGHBORHOOD_WATCH_SURVEY_dup'].value_counts()

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_WATCH'] != hf_neighbor_survey['NEIGHBORHOOD_WATCH_SURVEY_dup']][['person_id','NEIGHBORHOOD_WATCH','NEIGHBORHOOD_WATCH_SURVEY']]

#### get along

In [ ]:
hf_neighbor_survey[hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG'] != hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG_SURVEY']][['person_id','NEIGHBORHOOD_GET_ALONG','NEIGHBORHOOD_GET_ALONG_SURVEY']]

In [ ]:
print(hf_neighbor_survey[(hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG'] == 0) & (hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG_SURVEY'] == 4)].shape[0])
print(hf_neighbor_survey[(hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG'] == 1) & (hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG_SURVEY'] == 3)].shape[0])
print(hf_neighbor_survey[(hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG'] == 2) & (hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG_SURVEY'] == 2)].shape[0])
print(hf_neighbor_survey[(hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG'] == 4) & (hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG_SURVEY'] == 0)].shape[0])
print(hf_neighbor_survey[(hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG'] == 3) & (hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG_SURVEY'] == 1)].shape[0])
print(hf_neighbor_survey[(hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG'].isna() == True) & (hf_neighbor_survey['NEIGHBORHOOD_GET_ALONG_SURVEY'].isna() == True)].shape[0])
print(len(hf_neighbor_survey.index))

### income

**survey column = observation column**

In [ ]:
hf_education_survey[hf_education_survey['INCOME'] != hf_education_survey['INCOME_ANNUAL_SURVEY']]['INCOME'].unique()

### education

**higher sample size in survey column than in observation columns**

In [ ]:
print(len(hf_education_survey['EDUCATION_HIGHEST'].dropna().index))
print(len(hf_education_survey['EDUCATION_HIGHEST_SURVEY'].dropna().index))
print(len(hf_education_survey['EDUCATION_YEARS'].dropna().index))

# clean up columns

## drop duplicate columns between observation and survey data
- keep neighborhood observation columns
- keep income obeservation column
- drop these columns due to low numbers
    - EDUCATION_YEARS
    - EDUCATION_HIGHEST
- drop HF date- no longer needed

In [ ]:
hf_ses_sub = hf_ses.drop(columns = ['NEIGHBORHOOD_HOUSING_SURVEY',
                                    'NEIGHBORHOOD_SAFE_SURVEY',
                                    'NEIGHBORHOOD_CLEAN_SURVEY',
                                    'NEIGHBORHOOD_NOISE_SURVEY',
                                    'NEIGHBORHOOD_VANDALISM_SURVEY',
                                    'NEIGHBORHOOD_TRUST_SURVEY',
                                    'NEIGHBORHOOD_ALOT_CRIME_SURVEY',
                                    'NEIGHBORHOOD_GRAFFITI_SURVEY',
                                    'NEIGHBORHOOD_DRUG_USE_SURVEY',
                                    'NEIGHBORHOOD_SAME_VALUES_SURVEY',
                                    'NEIGHBORHOOD_ALCOHOL_SURVEY',
                                    'NEIGHBORHOOD_SIDEWALK_SURVEY',
                                    'NEIGHBORHOOD_WATCH_SURVEY',
                                    'NEIGHBORHOOD_BUILDING_SURVEY',
                                    'NEIGHBORHOOD_GET_ALONG_SURVEY',
                                    'NEIGHBORHOOD_UNSAFE_WALK_SURVEY',
                                    'NEIGHBORHOOD_CRIME_WALK_SURVEY',
                                    'NEIGHBORHOOD_CARE_SURVEY',
                                    'NEIGHBORHOOD_BIKE_SURVEY',
                                    'NEIGHBORHOOD_FREE_AMENITIES_SURVEY',
                                    'NEIGHBORHOOD_PPL_HANGING_AROUND_SURVEY',
                                    'NEIGHBORHOOD_TROUBLE_SURVEY',
                                    'NEIGHBORHOOD_STORES_SURVEY',
                                    'NEIGHBORHOOD_TRANSIT_SURVEY',
                                    'EDUCATION_HIGHEST',
                                    'EDUCATION_YEARS',
                                    'INCOME_ANNUAL_SURVEY',
                                    'HF_DATE'])
hf_ses_sub.columns

## rename remaining survey columns

In [ ]:
hf_ses_sub.rename(columns = {'EDUCATION_HIGHEST_SURVEY' : 'EDUCATION_HIGHEST'}, inplace = True)
hf_ses_sub.columns

# merge with pgs df

## subset and rename pgs df

In [ ]:
pgs.rename(columns = {'IID' : 'person_id',
                      'Z_norm2' : 'PGS'}, inplace = True)
pgs.head()

## remove non-AOU ids

In [ ]:
pgs_filt = pgs[~pgs['person_id'].str.contains('NA|HG|SS')]
print(len(pgs_filt.index))

## convert dtype to int

In [ ]:
pgs_filt['person_id'] = pgs_filt['person_id'].astype(int)
pgs_filt['person_id'].dtype

## merge

In [ ]:
hf_pgs = hf_ses_sub.merge(pgs_filt, on = 'person_id', how = 'left')
print(len(hf_pgs.dropna(subset = ['PGS']).index))
print(len(hf_pgs.index))
hf_pgs.head()

## export

In [ ]:
hf_pgs.to_csv('HF_Clinical_PGS.phenotype.csv', index = None)

# examine and remove missingness

## create column list

In [ ]:
col_list = hf_pgs.columns.tolist()
remove_list = ['person_id', 'HF', 'PGS']
for col in remove_list:
    col_list.remove(col)
print(len(hf_pgs.index))
col_list

## get percent missingness

In [ ]:
df_list = []

for col in col_list:
    percent = ((len(hf_pgs[col].dropna().index)) / 406513) * 100
    df = pd.DataFrame(data = {'COLUMN' : [col],
                              'PERCENT' : [percent]})
    df_list.append(df)

percent_df = pd.concat(df_list, axis = 0)
percent_df

## export percent missingness df

In [ ]:
percent_df.to_csv('HF.risk_factors.percent_missingness.csv', index = None)

## drop missing

In [ ]:
hf_pgs_no_missing = hf_pgs.dropna()
print(len(hf_pgs_no_missing.index))
hf_pgs_no_missing['HF'].value_counts(dropna = False)

## export df

In [ ]:
hf_pgs_no_missing.to_csv('HF_Clinical_PGS.phenotype.no_missing', index = None)

# normalize variables

## define inverse normal transformation function

In [ ]:
def inverse_normal_transform(x):
    """Applies rank-based inverse normal transformation."""
    ranks = x.rank(method='average', na_option='keep')
    n = ranks.notna().sum()
    transformed = norm.ppf((ranks - 0.5) / n)
    return transformed

## define min/max scaler function

In [ ]:
crs_scaler = MinMaxScaler()

In [ ]:
pxs_scaler =  MinMaxScaler(feature_range=(0, 2))

## CRS risk factors

### trig

In [ ]:
sns.kdeplot(hf_pgs_no_missing['TRIG'], fill = True)
plt.show()

In [ ]:
hf_pgs_no_missing['TRIG_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['TRIG'])
sns.kdeplot(hf_pgs_no_missing['TRIG_INV_NORMAL'], fill = True)
plt.show()
hf_pgs_no_missing['TRIG_INV_NORMAL_SCALE'] = crs_scaler.fit_transform(hf_pgs_no_missing[['TRIG_INV_NORMAL']])
sns.kdeplot(hf_pgs_no_missing['TRIG_INV_NORMAL_SCALE'], fill = True)
plt.show()

### LDL

In [ ]:
sns.kdeplot(hf_pgs_no_missing['LDL'], fill = True)
plt.show()

In [ ]:
hf_pgs_no_missing['LDL_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['LDL'])
sns.kdeplot(hf_pgs_no_missing['LDL_INV_NORMAL'], fill = True)
plt.show()
hf_pgs_no_missing['LDL_INV_NORMAL_SCALE'] = crs_scaler.fit_transform(hf_pgs_no_missing[['LDL_INV_NORMAL']])
sns.kdeplot(hf_pgs_no_missing['LDL_INV_NORMAL_SCALE'], fill = True)
plt.show()

### hdl

In [ ]:
hf_pgs_no_missing['HDL_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['HDL'])
hf_pgs_no_missing['HDL_INV_NORMAL_SCALE'] = crs_scaler.fit_transform(hf_pgs_no_missing[['HDL_INV_NORMAL']])

### glucose

In [ ]:
hf_pgs_no_missing['GLUCOSE_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['GLUCOSE'])
hf_pgs_no_missing['GLUCOSE_INV_NORMAL_SCALE'] = crs_scaler.fit_transform(hf_pgs_no_missing[['GLUCOSE_INV_NORMAL']])

### hba1c

In [ ]:
hf_pgs_no_missing['HbA1c_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['HbA1c'])
hf_pgs_no_missing['HbA1c_INV_NORMAL_SCALE'] = crs_scaler.fit_transform(hf_pgs_no_missing[['HbA1c_INV_NORMAL']])

### systolic

In [ ]:
hf_pgs_no_missing['SBP_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['SBP'])
hf_pgs_no_missing['SBP_INV_NORMAL_SCALE'] = crs_scaler.fit_transform(hf_pgs_no_missing[['SBP_INV_NORMAL']])

### diastolic

In [ ]:
hf_pgs_no_missing['DBP_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['DBP'])
hf_pgs_no_missing['DBP_INV_NORMAL_SCALE'] = crs_scaler.fit_transform(hf_pgs_no_missing[['DBP_INV_NORMAL']])

## PXS risk factors

### bmi

In [ ]:
hf_pgs_no_missing['BMI_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['BMI'])
hf_pgs_no_missing['BMI_INV_NORMAL_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['BMI_INV_NORMAL']])
sns.kdeplot(hf_pgs_no_missing['BMI_INV_NORMAL_SCALE'], fill = True)
plt.show()

### physical activity

In [ ]:
counts = hf_pgs_no_missing['PA_EVERYDAY'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()
hf_pgs_no_missing['PA_EVERYDAY_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['PA_EVERYDAY']])
counts = hf_pgs_no_missing['PA_EVERYDAY_SCALE'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()

### neighborhood

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_DRUG_USE_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_DRUG_USE']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_SAFE_CRIME_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_SAFE_CRIME']])

In [ ]:
counts = hf_pgs_no_missing['NEIGHBORHOOD_TRUST'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()
hf_pgs_no_missing['NEIGHBORHOOD_TRUST_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_TRUST']])
counts = hf_pgs_no_missing['NEIGHBORHOOD_TRUST_SCALE'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_BUILDING_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_BUILDING']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_ALCOHOL_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_ALCOHOL']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_VANDALISM_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_VANDALISM']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_SIDEWALK_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_SIDEWALK']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_BIKE_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_BIKE']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_CLEAN_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_CLEAN']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_WATCH_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_WATCH']])

In [ ]:
counts = hf_pgs_no_missing['NEIGHBORHOOD_HOUSING'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()
hf_pgs_no_missing['NEIGHBORHOOD_HOUSING_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_HOUSING']])
counts = hf_pgs_no_missing['NEIGHBORHOOD_HOUSING_SCALE'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()

In [ ]:
counts = hf_pgs_no_missing['NEIGHBORHOOD_GET_ALONG'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()
hf_pgs_no_missing['NEIGHBORHOOD_GET_ALONG_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_GET_ALONG']])
counts = hf_pgs_no_missing['NEIGHBORHOOD_GET_ALONG_SCALE'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_UNSAFE_WALK_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_UNSAFE_WALK']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_CARE_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_CARE']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_ALOT_CRIME_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_ALOT_CRIME']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_CRIME_WALK_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_CRIME_WALK']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_SAME_VALUES_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_SAME_VALUES']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_GRAFFITI_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_GRAFFITI']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_NOISE_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_NOISE']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_FREE_AMENITIES_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_FREE_AMENITIES']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_PPL_HANGING_AROUND_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_PPL_HANGING_AROUND']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_TROUBLE_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_TROUBLE']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_STORES_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_STORES']])

In [ ]:
hf_pgs_no_missing['NEIGHBORHOOD_TRANSIT_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['NEIGHBORHOOD_TRANSIT']])

### income

In [ ]:
counts = hf_pgs_no_missing['INCOME'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()
hf_pgs_no_missing['INCOME_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['INCOME']])
counts = hf_pgs_no_missing['INCOME_SCALE'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()

### highest education

In [ ]:
counts = hf_pgs_no_missing['EDUCATION_HIGHEST'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()
hf_pgs_no_missing['EDUCATION_HIGHEST_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['EDUCATION_HIGHEST']])
counts = hf_pgs_no_missing['EDUCATION_HIGHEST_SCALE'].value_counts().sort_index()
plt.bar(counts.index, counts.values)
plt.show()

### census median income

In [ ]:
sns.kdeplot(hf_pgs_no_missing['CENSUS_MEDIAN_INCOME'], fill=True)
plt.show()
hf_pgs_no_missing['CENSUS_MEDIAN_INCOME_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['CENSUS_MEDIAN_INCOME'])
sns.kdeplot(hf_pgs_no_missing['CENSUS_MEDIAN_INCOME_INV_NORMAL'], fill=True)
plt.show()
hf_pgs_no_missing['CENSUS_MEDIAN_INCOME_INV_NORMAL_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['CENSUS_MEDIAN_INCOME_INV_NORMAL']])
sns.kdeplot(hf_pgs_no_missing['CENSUS_MEDIAN_INCOME_INV_NORMAL_SCALE'], fill=True)
plt.show()

### social deprivation index

In [ ]:
sns.kdeplot(hf_pgs_no_missing['SOCIAL_DEPRIVATION_INDEX'], fill = True)
plt.show()
hf_pgs_no_missing['SOCIAL_DEPRIVATION_INDEX_INV_NORMAL'] = inverse_normal_transform(hf_pgs_no_missing['SOCIAL_DEPRIVATION_INDEX'])
sns.kdeplot(hf_pgs_no_missing['SOCIAL_DEPRIVATION_INDEX_INV_NORMAL'], fill = True)
plt.show()
hf_pgs_no_missing['SOCIAL_DEPRIVATION_INDEX_INV_NORMAL_SCALE'] = pxs_scaler.fit_transform(hf_pgs_no_missing[['SOCIAL_DEPRIVATION_INDEX_INV_NORMAL']])
sns.kdeplot(hf_pgs_no_missing['SOCIAL_DEPRIVATION_INDEX_INV_NORMAL_SCALE'], fill=True)
plt.show()

## export df

In [ ]:
hf_pgs_no_missing.to_csv('HF_Clinical_PGS.phenotype.no_missing.variable_transformation.csv', index = None)

# read in outputs from feature selection 1000 iterations

## read in and combine outputs

In [ ]:
significant = []
insignificant = []
important = []
unimportant = []
beta = []
pval = []
coef = []

for iter in list(range(1, 1001)):
    significant_filename = 'train/LR_significant_ITER_' + str(iter) + '.txt'
    insignificant_filename = 'train/LR_insignificant_ITER_' + str(iter) + '.txt'
    beta_filename = 'train/LR_beta_ITER_' + str(iter) + '.txt'
    pval_filename = 'train/LR_pval_ITER_' + str(iter) + '.txt'
    important_filename = 'train/LASSO_important_ITER_' + str(iter) + '.txt'
    unimportant_filename = 'train/LASSO_unimportant_ITER_' + str(iter) + '.txt'
    coef_filename = 'train/LASSO_coef_ITER_' + str(iter) + '.txt'
    
    significant.append(pd.read_csv(significant_filename, sep = '\t', index_col = 0))
    insignificant.append(pd.read_csv(insignificant_filename, sep = '\t', index_col = 0))
    important.append(pd.read_csv(important_filename, sep = '\t', index_col = 0))
    unimportant.append(pd.read_csv(unimportant_filename, sep = '\t', index_col = 0))
    beta.append(pd.read_csv(beta_filename, sep = '\t', index_col = 0))
    pval.append(pd.read_csv(pval_filename, sep = '\t', index_col = 0))
    coef.append(pd.read_csv(coef_filename, sep = '\t', index_col = 0))

significant_df = pd.concat(significant, axis = 1)
insignificant_df = pd.concat(insignificant, axis = 1)
important_df = pd.concat(important, axis = 1)
unimportant_df = pd.concat(unimportant, axis = 1)
beta_df = pd.concat(beta, axis = 1)
pval_df = pd.concat(pval, axis = 1)
coef_df = pd.concat(coef, axis = 1)

## identify number/percent significant

In [ ]:
significant_df['num_sig'] = significant_df.notna().sum(axis = 1)
significant_df['percent_sig'] = (significant_df['num_sig']/1000) * 100
significant_df[['num_sig', 'percent_sig']]

## identify number/percent insignificant

In [ ]:
insignificant_df['num_insig'] = insignificant_df.notna().sum(axis = 1)
insignificant_df['percent_insig'] = (insignificant_df['num_insig']/1000) * 100
insignificant_df[['num_insig', 'percent_insig']]

## identify number/percent important

In [ ]:
important_df['num_important'] = important_df.notna().sum(axis = 1)
important_df['percent_important'] = (important_df['num_important']/1000) * 100
important_df[['num_important', 'percent_important']]

## identify number/percent unimportant

In [ ]:
unimportant_df['num_unimportant'] = unimportant_df.isna().sum(axis = 1)
unimportant_df['percent_unimportant'] = (unimportant_df['num_unimportant']/1000) * 100
unimportant_df[['num_unimportant', 'percent_unimportant']]

## calculate mean beta

In [ ]:
beta_df['MEAN_BETA'] = beta_df.mean(axis = 1)
beta_df[['MEAN_BETA']]

## calculate mean pval

In [ ]:
pval_df['MEAN_PVAL'] = pval_df.mean(axis = 1)
pval_df[['MEAN_PVAL']]

## calculate mean absolute coefficient

In [ ]:
coef_df['MEAN_COEF'] = coef_df.abs().mean(axis = 1)
coef_df[['MEAN_COEF']]

## filter columns to sig and important >= 95% time

In [ ]:
significant_95 = significant_df[significant_df['percent_sig'] >= 95]
significant_95[['num_sig', 'percent_sig']]

In [ ]:
important_95 = important_df[important_df['percent_important'] >= 95]
important_95[['num_important', 'percent_important']]

In [ ]:
print(len(significant_95.index.intersection(important_95.index)))
significant_95.index.intersection(important_95.index)

## combine aggregate metrics into one df

In [ ]:
sig_sub = significant_df[['percent_sig']]
sig_sub.rename(columns = {'percent_sig' : 'PERCENT_SIGNIFICANT'}, inplace = True)
important_sub = important_df[['percent_important']]
important_sub.rename(columns = {'percent_important' : 'PERCENT_IMPORTANT'}, inplace = True)
pval_sub = pval_df[['MEAN_PVAL']]
coef_sub = coef_df[['MEAN_COEF']]
beta_sub = beta_df[['MEAN_BETA']]
sig_important = pd.concat([sig_sub, pval_sub, important_sub, coef_sub, beta_sub], axis = 1)
sig_important

## export

In [ ]:
significant_95_sub = significant_95[['num_sig', 'percent_sig']]
significant_95.to_csv('significant_vars_95.csv')

In [ ]:
important_95_sub = important_95[['num_important', 'percent_important']]
important_95.to_csv('important_vars_95.csv')

In [ ]:
beta_df_export = beta_df.drop(columns = ['MEAN_BETA'])
beta_df_export.to_csv('LR_beta_all_iter.csv')

In [ ]:
sig_important.to_csv('all_vars_significant_important.csv')

# read in outputs from model evaluation 1000 iterations

## read in and combine outputs

In [ ]:
auroc = []
auprc = []
f1 = []
balanced_acc = []

for iter in list(range(1, 1001)):
    auroc_filename = 'eval/AUROC_ITER_' + str(iter) + '.txt'
    auprc_filename = 'eval/AUPRC_ITER_' + str(iter) + '.txt'
    f1_filename = 'eval/F1_SCORE_ITER_' + str(iter) + '.txt'
    balanced_acc_filename = 'eval/BALANCED_ACCURACY_ITER_' + str(iter) + '.txt'
    
    auroc.append(pd.read_csv(auroc_filename, sep = '\t', index_col = 0))
    auprc.append(pd.read_csv(auprc_filename, sep = '\t', index_col = 0))
    f1.append(pd.read_csv(f1_filename, sep = '\t', index_col = 0))
    balanced_acc.append(pd.read_csv(balanced_acc_filename, sep = '\t', index_col = 0))

auroc_df = pd.concat(auroc, axis = 1)
auprc_df = pd.concat(auprc, axis = 1)
f1_df = pd.concat(f1, axis = 1)
balanced_acc_df = pd.concat(balanced_acc, axis = 1)

## calculate mean metrics

In [ ]:
auroc_df['AUROC'] = auroc_df.mean(axis = 1)
auroc_df_sub = auroc_df[['AUROC']]

In [ ]:
auprc_df['AUPRC'] = auprc_df.mean(axis = 1)
auprc_df_sub = auprc_df[['AUPRC']]

In [ ]:
f1_df['F1_SCORE'] = f1_df.mean(axis = 1)
f1_df_sub = f1_df[['F1_SCORE']]

In [ ]:
balanced_acc_df['BALANCED_ACCURACY'] = balanced_acc_df.mean(axis = 1)
balanced_acc_df_sub = balanced_acc_df[['BALANCED_ACCURACY']]

## combine mean metrics into one df

In [ ]:
all_metrics = pd.concat([auroc_df_sub, auprc_df_sub, f1_df_sub, balanced_acc_df_sub], axis = 1)
all_metrics

## reformat model names

In [ ]:
all_metrics.index = all_metrics.index.map(lambda x: x.replace("'", ""))
all_metrics.index = all_metrics.index.map(lambda x: x.replace("(", ""))
all_metrics.index = all_metrics.index.map(lambda x: x.replace(")", ""))
all_metrics.index = all_metrics.index.map(lambda x: x.replace(",", " +"))
all_metrics.index = all_metrics.index.map(lambda x: x.replace("PGS + T2D + LDL_INV_NORMAL_SCALE + HDL_INV_NORMAL_SCALE + BMI_INV_NORMAL_SCALE + SMOKING + PA_EVERYDAY_SCALE + NEIGHBORHOOD_DRUG_USE_SCALE + NEIGHBORHOOD_SAFE_CRIME_SCALE + NEIGHBORHOOD_TRUST_SCALE + NEIGHBORHOOD_BUILDING_SCALE + NEIGHBORHOOD_VANDALISM_SCALE + NEIGHBORHOOD_SIDEWALK_SCALE + NEIGHBORHOOD_BIKE_SCALE + NEIGHBORHOOD_CLEAN_SCALE + NEIGHBORHOOD_UNSAFE_WALK_SCALE + NEIGHBORHOOD_CARE_SCALE + NEIGHBORHOOD_ALOT_CRIME_SCALE + NEIGHBORHOOD_CRIME_WALK_SCALE + NEIGHBORHOOD_GRAFFITI_SCALE + NEIGHBORHOOD_FREE_AMENITIES_SCALE + NEIGHBORHOOD_PPL_HANGING_AROUND_SCALE + NEIGHBORHOOD_TROUBLE_SCALE + NEIGHBORHOOD_STORES_SCALE + NEIGHBORHOOD_TRANSIT_SCALE + INCOME_SCALE + EDUCATION_HIGHEST_SCALE + CENSUS_MEDIAN_INCOME_INV_NORMAL_SCALE",
                                                              "PGS + CRS Risk Factors + PXS Risk Factors"))
all_metrics.index = all_metrics.index.map(lambda x: x.replace("T2D + LDL_INV_NORMAL_SCALE + HDL_INV_NORMAL_SCALE + BMI_INV_NORMAL_SCALE + SMOKING + PA_EVERYDAY_SCALE + NEIGHBORHOOD_DRUG_USE_SCALE + NEIGHBORHOOD_SAFE_CRIME_SCALE + NEIGHBORHOOD_TRUST_SCALE + NEIGHBORHOOD_BUILDING_SCALE + NEIGHBORHOOD_VANDALISM_SCALE + NEIGHBORHOOD_SIDEWALK_SCALE + NEIGHBORHOOD_BIKE_SCALE + NEIGHBORHOOD_CLEAN_SCALE + NEIGHBORHOOD_UNSAFE_WALK_SCALE + NEIGHBORHOOD_CARE_SCALE + NEIGHBORHOOD_ALOT_CRIME_SCALE + NEIGHBORHOOD_CRIME_WALK_SCALE + NEIGHBORHOOD_GRAFFITI_SCALE + NEIGHBORHOOD_FREE_AMENITIES_SCALE + NEIGHBORHOOD_PPL_HANGING_AROUND_SCALE + NEIGHBORHOOD_TROUBLE_SCALE + NEIGHBORHOOD_STORES_SCALE + NEIGHBORHOOD_TRANSIT_SCALE + INCOME_SCALE + EDUCATION_HIGHEST_SCALE + CENSUS_MEDIAN_INCOME_INV_NORMAL_SCALE",
                                                              "CRS Risk Factors + PXS Risk Factors"))
all_metrics.index = all_metrics.index.map(lambda x: x.replace("T2D + LDL_INV_NORMAL_SCALE + HDL_INV_NORMAL_SCALE",
                                                              "CRS Risk Factors"))
all_metrics.index = all_metrics.index.map(lambda x: x.replace("BMI_INV_NORMAL_SCALE + SMOKING + PA_EVERYDAY_SCALE + NEIGHBORHOOD_DRUG_USE_SCALE + NEIGHBORHOOD_SAFE_CRIME_SCALE + NEIGHBORHOOD_TRUST_SCALE + NEIGHBORHOOD_BUILDING_SCALE + NEIGHBORHOOD_VANDALISM_SCALE + NEIGHBORHOOD_SIDEWALK_SCALE + NEIGHBORHOOD_BIKE_SCALE + NEIGHBORHOOD_CLEAN_SCALE + NEIGHBORHOOD_UNSAFE_WALK_SCALE + NEIGHBORHOOD_CARE_SCALE + NEIGHBORHOOD_ALOT_CRIME_SCALE + NEIGHBORHOOD_CRIME_WALK_SCALE + NEIGHBORHOOD_GRAFFITI_SCALE + NEIGHBORHOOD_FREE_AMENITIES_SCALE + NEIGHBORHOOD_PPL_HANGING_AROUND_SCALE + NEIGHBORHOOD_TROUBLE_SCALE + NEIGHBORHOOD_STORES_SCALE + NEIGHBORHOOD_TRANSIT_SCALE + INCOME_SCALE + EDUCATION_HIGHEST_SCALE + CENSUS_MEDIAN_INCOME_INV_NORMAL_SCALE",
                                                              "PXS Risk Factors"))

all_metrics

In [ ]:
row_order = ['PGS',
            'CRS_SUM',
            'CRS_WEIGHTED_SUM',
            'PXS_SUM',
            'PXS_WEIGHTED_SUM',
            'PGS + CRS_SUM',
            'PGS + CRS_WEIGHTED_SUM',
            'PGS + PXS_SUM',
            'PGS + PXS_WEIGHTED_SUM',
            'CRS_SUM + PXS_SUM',
            'CRS_WEIGHTED_SUM + PXS_WEIGHTED_SUM',
            'PGS + CRS_SUM + PXS_SUM',
            'PGS + CRS_WEIGHTED_SUM + PXS_WEIGHTED_SUM',
            'CRS Risk Factors',
            'PXS Risk Factors',
            'CRS Risk Factors + PXS Risk Factors',
            'PGS + CRS Risk Factors + PXS Risk Factors']

In [ ]:
all_metrics = all_metrics.reindex(row_order)
all_metrics

In [ ]:
all_metrics.index = [f'Model {i + 1}: {idx}' for i, idx in enumerate(all_metrics.index)]
all_metrics